In [1036]:
import os
import numpy as np
import pandas as pd
from math import ceil
from collections import deque
from itertools import combinations
import math
import sys
global maxdays
current_directory = os.getcwd()
%cd \Users\\Sebas\\OneDrive\\Documenten\\School\\3e jaar\\Project CO


C:\Users\Sebas\OneDrive\Documenten\School\3e jaar\Project CO


In [1037]:
directory = "\\Users\\Sebas\\OneDrive\\Documenten\\School\\3e jaar\\Project CO"
filename = 'CO_Case2419.txt'
output_file_name = "CO_Case2419_sol.txt"

"""
 Read the instance and assign variables to the different values
"""

data = pd.read_csv(filename, sep='\t+', header=None, engine='python')
data = data[0].str.split(expand=True)
title = data.iloc[1,2] + ' ' + data.iloc[1,3]
maxdays = data.iloc[2,2]
maxdays = int(maxdays)
constraints = data[[0, 2]].iloc[2:5]
constraints = constraints.set_index(0)[2].to_dict()
constraints = dict((k, int(v)) for k, v in constraints.items())
costs = data[[0, 2]].iloc[5:11]
costs = costs.set_index(0)[2].to_dict()
costs = dict((k, int(v)) for k, v in costs.items())
numberMachines = int(data.iloc[11][2])
index = (11 + numberMachines + 1)  # extra step that ensures the right values are taken
machines = np.array(data[[0, 1, 2]][12:index].astype(int))
numberLocations = int(data.iloc[index][2])
locations = np.array(data[[0, 1, 2]][index + 1:(index + numberLocations + 1)].astype(int))
index += numberLocations + 1  # extra step that ensures the right values are taken
depot = locations[0]
numberRequests = int(data.iloc[index][2])
requests = np.array(data[[0, 1, 2, 3, 4, 5]][index + 1:(index + numberRequests + 1)].astype(int))
index += numberRequests + 1  # extra step that ensures the right values are taken
numberTechnicians = int(data.iloc[index][2])
technicians = np.array(
        data[[i for i in range(0, numberMachines + 4)]][index + 1:(index + numberTechnicians + 1)].astype(int))
constraints

DAYS = constraints['DAYS']
TRUCK_CAPACITY = constraints['TRUCK_CAPACITY']
TRUCK_MAX_DISTANCE = constraints['TRUCK_MAX_DISTANCE']

TRUCK_DISTANCE_COST = costs['TRUCK_DISTANCE_COST']
TRUCK_DAY_COST = costs['TRUCK_DAY_COST']
TRUCK_COST = costs['TRUCK_COST']
TECHNICIAN_DISTANCE_COST = costs['TECHNICIAN_DISTANCE_COST']
TECHNICIAN_DAY_COST = costs['TECHNICIAN_DAY_COST']
TECHNICIAN_COST = costs['TECHNICIAN_COST']
IDLE_MACHINE_COSTS = {machine[0]: machine[2] for machine in machines}


In [1038]:
def nearest_neighbor(point, unvisited_points, locations):
    #Find the nearest neighbor of a given point among a list of unvisited points. Returns nearest_point: The nearest neighbor of the given point.
    min_distance = float('inf')
    nearest_point = None
    point_coordinates = get_location_coordinates(point, locations)
    
    for candidate_point in unvisited_points:
        candidate_point_id = int(candidate_point)
        if candidate_point_id != int(point):
            candidate_coordinates = get_location_coordinates(candidate_point_id, locations)
            dist = calculate_distance(point_coordinates, candidate_coordinates)
            if dist is not None and dist < min_distance:
                min_distance = dist
                nearest_point = candidate_point_id
    return nearest_point

def get_location_coordinates(location_id, locations):
    #Get the coordinates of a location based on its ID. Returns coordinates: Tuple (x, y) representing the coordinates of the location.

    for location in locations:
        if location[0] == location_id:
            return (location[1], location[2])
    return None

def calculate_distance(coordinates1, coordinates2):
    #Calculate the Euclidean distance between two points given their coordinates and round up to the nearest integer using the ceil function. Returns
    #distance: The Euclidean distance between the two points, rounded up to the nearest integer.
    x1, y1 = coordinates1
    x2, y2 = coordinates2
    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return ceil(distance)

def get_location_details(requests, locations, request_id):
    #Get the location details (ID and coordinates) corresponding to the given request ID.Returns location_details as a tuple 
    #[(location_id, x_coordinate, y_coordinate)]
  
    for request in requests:
        if request[0] == request_id:  # Check if request ID matches
            location_id = request[1]
            for location in locations:
                if location[0] == location_id:  # Find location details by matching location ID
                    return (location_id, location[1], location[2])  # Return location details
    return None  # Return None if request ID not found

def calculate_request_distance(requests, locations, request_id1, request_id2):
    #Calculate the distance between two requests based on their corresponding location coordinates. Returns the Euclidean distance.

    location1 = get_location_details(requests, locations, request_id1)
    location2 = get_location_details(requests, locations, request_id2)
    
    if location1 and location2:
        coordinates1 = (location1[1], location1[2])
        coordinates2 = (location2[1], location2[2])
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None

def get_requests_for_location(requests, location_id):
    #Get the list of request IDs corresponding to the given location ID. 

    request_ids = [req[0] for req in requests if req[1] == location_id]
    return request_ids

unique_location_ids = set(req[1] for req in requests)

# Iterate over unique location IDs
for location_id in unique_location_ids:
    request_ids_for_location = get_requests_for_location(requests, location_id)
  
    
def group_requests_by_location(requests):
    #Group requests by their location ID.Returns a dictionary where keys are location IDs and values are lists of request IDs.

    location_groups = {}
    for request in requests:
        request_id, location_id = request[0], request[1]
        if location_id not in location_groups:
            location_groups[location_id] = []
        location_groups[location_id].append(request_id)
    return location_groups

location_groups = group_requests_by_location(requests)

def calculate_request_weight(request_id, requests, machines):
    request = [req for req in requests if req[0] == request_id][0]
    machine_kind_id = request[4]
    num_requested_machines = request[5]
    machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
    return machine_size * num_requested_machines

def check_truck_capacity(locations, depot, TRUCK_CAPACITY, requests, machines):
    depot_loc_ID = depot[0]
    truck_loads = []
    current_truck_load = []
    current_truck_weight = 0
    
    for request_id, location_id, _, _, machine_kind_id, num_requested_machines in requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        if current_truck_weight + request_weight > TRUCK_CAPACITY:
            if current_truck_load:
                truck_loads.append(current_truck_load)
            current_truck_load = [(request_id, request_weight)]
            current_truck_weight = request_weight
        else:
            current_truck_load.append((request_id, request_weight))
            current_truck_weight += request_weight
    

In [1039]:
def get_machine_type_from_request(requests):
    #Get the machine type from each request and store it in a dictionary where keys are request IDs and values are machine types.
    machine_types = {}
    for request in requests:
        request_id = request[0]
        machine_type = request[4]  
        machine_types[request_id] = machine_type

    return machine_types

machine_types = get_machine_type_from_request(requests)

def get_penalty_cost_from_machine(machines):
    #Extract the penalty cost from each request. Returns dictionary where keys are request IDs and values are the penalty costs.
    penalty_cost = {}
    for mach in machines:
        penalty_cost[mach[0]] = mach[2]  
    return penalty_cost

penalty_costs_machines = get_penalty_cost_from_machine(machines)


In [1040]:
def calculate_depot_to_request_distance(locations, depot_loc_ID, request_id):
    #Calculate the Euclidean distance between the depot and a specific request based on their corresponding location coordinates.

    depot_coordinates = get_location_coordinates(depot_loc_ID, locations)
    request_location_id = [request[1] for request in requests if request[0] == request_id]
    if request_location_id:
        request_coordinates = get_location_coordinates(request_location_id[0], locations)
        distance = calculate_distance(depot_coordinates, request_coordinates)
        return distance
    else:
        return None

# Calculate the distances between the depot and each request
depot_loc_ID = 1  # Assuming the depot location ID is always 1
depot_to_requests = {}
for request_id in range(1, len(requests) + 1):
    distance = calculate_depot_to_request_distance(locations, depot_loc_ID, request_id)
    if distance is not None:
        depot_to_requests[request_id] = distance

def distance_between_consecutive_requests(locations, requests, request_id1, request_id2):
    #Calculate the Euclidean distance between the two consecutive requests.

    location_id1 = None
    location_id2 = None

    for request in requests:
        if request[0] == request_id1:
            location_id1 = request[1]
        elif request[0] == request_id2:
            location_id2 = request[1]

    # If both location IDs are found, calculate the distance
    if location_id1 is not None and location_id2 is not None:
        coordinates1 = get_location_coordinates(location_id1, locations)
        coordinates2 = get_location_coordinates(location_id2, locations)
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None

def calculate_consecutive_request_distances(locations, requests):
    #Calculate and store the distances between all pairs of consecutive requests. Returns dictionary storing the distances between consecutive requests.

    consecutive_distances = {}

    # Iterate through the requests to calculate distances between consecutive pairs
    for i in range(len(requests) - 1):
        request_id1 = requests[i][0]
        request_id2 = requests[i + 1][0]
        distance = distance_between_consecutive_requests(locations, requests, request_id1, request_id2)
        if distance is not None:
            consecutive_distances[(request_id1, request_id2)] = distance

    return consecutive_distances

# Calculate and store the distances between consecutive requests
consecutive_distances = calculate_consecutive_request_distances(locations, requests)


In [1041]:
def combined_requests_from_and_to_depot_distance(locations, requests, depot_loc_ID, request_id1, request_id2):
    #Calculate the combined distances from the depot to the first request, between the two requests, and from the second request back to the depot.
    #Returns dictionary storing the combined distances.

    combined_distances = {}
    depot_to_first_request_distance = calculate_depot_to_request_distance(locations, depot_loc_ID, request_id1)
    consecutive_requests_distance = distance_between_consecutive_requests(locations, requests, request_id1, request_id2)
    second_request_to_depot_distance = calculate_depot_to_request_distance(locations, depot_loc_ID, request_id2)
    combined_distances[(request_id1, request_id2)] = {
        'depot_to_first_request_distance': depot_to_first_request_distance,
        'consecutive_requests_distance': consecutive_requests_distance,
        'second_request_to_depot_distance': second_request_to_depot_distance
    }
    
    return combined_distances

def calculate_all_combined_distances(locations, requests, depot_loc_ID):
    """
    Calculate the combined distances from the depot to the first request, between the two requests,
    and from the second request back to the depot for all pairs of requests.
    Returns dictionary storing the combined distances for all pairs of requests.
    """
    combined_distances = {}

    # Iterate through all possible pairs of requests
    for i in range(len(requests)):
        request_id1 = requests[i][0]
        for j in range(i + 1, len(requests)):
            request_id2 = requests[j][0]

            # Calculate the combined distances for the current pair of requests
            depot_to_first_request_distance = calculate_depot_to_request_distance(locations, depot_loc_ID, request_id1)
            consecutive_requests_distance = distance_between_consecutive_requests(locations, requests, request_id1, request_id2)
            second_request_to_depot_distance = calculate_depot_to_request_distance(locations, depot_loc_ID, request_id2)

            # Store the combined distances in the dictionary
            combined_distances[(request_id1, request_id2)] = {
                'depot_to_first_request_distance': depot_to_first_request_distance,
                'consecutive_requests_distance': consecutive_requests_distance,
                'second_request_to_depot_distance': second_request_to_depot_distance
            }

    return combined_distances

# Create a dictionary to store total distances for each pair
total_distances_for_pairs = {}

# Calculate and store the distances between all pairs of requests
combined_distances = calculate_all_combined_distances(locations, requests, depot_loc_ID)

# Iterate through the dictionary and calculate the total distance for each pair
for pair, distances in combined_distances.items():
    total_distance_for_pair = (
        distances['depot_to_first_request_distance']
        + distances['consecutive_requests_distance']
        + distances['second_request_to_depot_distance']
    )
    total_distances_for_pairs[pair] = total_distance_for_pair

# Print or access the dictionary as needed
print(total_distances_for_pairs)



{(1, 2): 1143, (1, 3): 754, (1, 4): 754, (1, 5): 759, (1, 6): 759, (1, 7): 1143, (1, 8): 759, (1, 9): 860, (1, 10): 690, (1, 11): 754, (1, 12): 860, (1, 13): 759, (1, 14): 759, (1, 15): 860, (1, 16): 690, (1, 17): 690, (1, 18): 690, (1, 19): 860, (1, 20): 754, (1, 21): 614, (1, 22): 1143, (1, 23): 690, (1, 24): 860, (1, 25): 1143, (1, 26): 759, (1, 27): 754, (1, 28): 759, (1, 29): 860, (1, 30): 754, (2, 3): 1184, (2, 4): 1184, (2, 5): 955, (2, 6): 955, (2, 7): 544, (2, 8): 955, (2, 9): 933, (2, 10): 1198, (2, 11): 1184, (2, 12): 933, (2, 13): 955, (2, 14): 955, (2, 15): 933, (2, 16): 1198, (2, 17): 1198, (2, 18): 1198, (2, 19): 933, (2, 20): 1184, (2, 21): 1143, (2, 22): 544, (2, 23): 1198, (2, 24): 933, (2, 25): 544, (2, 26): 955, (2, 27): 1184, (2, 28): 955, (2, 29): 933, (2, 30): 1184, (3, 4): 682, (3, 5): 740, (3, 6): 740, (3, 7): 1184, (3, 8): 740, (3, 9): 835, (3, 10): 751, (3, 11): 682, (3, 12): 835, (3, 13): 740, (3, 14): 740, (3, 15): 835, (3, 16): 751, (3, 17): 751, (3, 18): 

In [1042]:
def check_feasibility_for_all_pairs(total_distances_for_pairs, TRUCK_MAX_DISTANCE):
    """
    Check the feasibility of total distances for all pairs.

    Args:
    - total_distances_for_pairs: Dictionary where keys are pairs and values are total distances.
    - MAX_TRUCK_CAPACITY: Maximum truck capacity.

    Returns:
    - feasibility_dict: Dictionary where keys are pairs and values are boolean indicating feasibility.
    """
    feasibility_dict = {}
    for pair, total_distance in total_distances_for_pairs.items():
        feasibility_dict[pair] = total_distance <= TRUCK_MAX_DISTANCE
    return feasibility_dict

# Check feasibility for all pairs
feasibility_dict = check_feasibility_for_all_pairs(total_distances_for_pairs, TRUCK_MAX_DISTANCE)
# Print feasibility for each pair
for pair, feasible in feasibility_dict.items():
    print(f"Pair {pair}: Feasible? {feasible}")




Pair (1, 2): Feasible? False
Pair (1, 3): Feasible? False
Pair (1, 4): Feasible? False
Pair (1, 5): Feasible? False
Pair (1, 6): Feasible? False
Pair (1, 7): Feasible? False
Pair (1, 8): Feasible? False
Pair (1, 9): Feasible? False
Pair (1, 10): Feasible? True
Pair (1, 11): Feasible? False
Pair (1, 12): Feasible? False
Pair (1, 13): Feasible? False
Pair (1, 14): Feasible? False
Pair (1, 15): Feasible? False
Pair (1, 16): Feasible? True
Pair (1, 17): Feasible? True
Pair (1, 18): Feasible? True
Pair (1, 19): Feasible? False
Pair (1, 20): Feasible? False
Pair (1, 21): Feasible? True
Pair (1, 22): Feasible? False
Pair (1, 23): Feasible? True
Pair (1, 24): Feasible? False
Pair (1, 25): Feasible? False
Pair (1, 26): Feasible? False
Pair (1, 27): Feasible? False
Pair (1, 28): Feasible? False
Pair (1, 29): Feasible? False
Pair (1, 30): Feasible? False
Pair (2, 3): Feasible? False
Pair (2, 4): Feasible? False
Pair (2, 5): Feasible? False
Pair (2, 6): Feasible? False
Pair (2, 7): Feasible? True


In [1043]:
def get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = []
    current_truck_weight = 0
    current_delivery_day = None
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        if not current_truck_route:
            current_truck_route.append([request_id])
            current_truck_weight = request_weight
            current_delivery_day = start_day
        elif current_truck_weight + request_weight <= TRUCK_CAPACITY and start_day == current_delivery_day:
            # Check if adding the current request exceeds the truck capacity and if delivery day matches
            combined_request_ids = current_truck_route[-1] + [request_id]
            combined_weight = current_truck_weight + request_weight
            if combined_weight <= TRUCK_CAPACITY:
                current_truck_route[-1] = combined_request_ids
                current_truck_weight = combined_weight
            else:
                truck_routes.append((current_truck_route, current_delivery_day))
                current_truck_route = [[request_id]]
                current_truck_weight = request_weight
                current_delivery_day = start_day
        else:
            # If adding the current request exceeds the truck capacity or delivery day does not match,
            # start a new truck route with the current request
            truck_routes.append((current_truck_route, current_delivery_day))
            current_truck_route = [[request_id]]
            current_truck_weight = request_weight
            current_delivery_day = start_day
    
    # Append the last truck route if it's not empty
    if current_truck_route:
        truck_routes.append((current_truck_route, current_delivery_day))
    
    return truck_routes


def calculate_delivery_days(truck_routes):
    delivery_days = {}
    for truck_route, delivery_day in truck_routes:
        for request_ids in truck_route:
            for request_id in request_ids:
                delivery_days[request_id] = delivery_day
    return delivery_days



# Calculate truck routes
truck_routes = get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines)

# Calculate delivery days using truck routes
delivery_days = calculate_delivery_days(truck_routes)

print(truck_routes)

def get_truck_routes_with_constraints(locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, requests, machines, feasibility_dict):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = []
    current_truck_weight = 0
    current_truck_distance = 0
    current_delivery_day = None
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        # Calculate the distance from the depot to the current location
        depot_to_location_distance = calculate_request_distance(requests, locations, depot_loc_ID, location_id)
        
        # Check if the distance calculation was successful
        if depot_to_location_distance is None:
            print(f"Error: Unable to calculate distance between depot and request {request_id}")
            continue
        
        # Check if adding the current request meets all constraints
        if (current_truck_weight + request_weight <= TRUCK_CAPACITY and 
            current_truck_distance + depot_to_location_distance <= TRUCK_MAX_DISTANCE and
            start_day == current_delivery_day):
            # If constraints are satisfied, add the request to the current truck route
            if current_truck_route:
                combined_request_ids = current_truck_route[-1] + [request_id]
                if all(feasibility_dict[(req_id1, req_id2)] for req_id1 in current_truck_route[-1] for req_id2 in [request_id]):
                    current_truck_route[-1] = combined_request_ids
                    current_truck_weight += request_weight
                    current_truck_distance += depot_to_location_distance
                else:
                    truck_routes.append((current_truck_route, current_delivery_day))
                    current_truck_route = [[request_id]]  # Start a new truck route
                    current_truck_weight = request_weight
                    current_truck_distance = depot_to_location_distance
                    current_delivery_day = start_day
            else:
                current_truck_route.append([request_id])  # Add the request ID
                current_truck_weight += request_weight
                current_truck_distance += depot_to_location_distance
        else:
            # If constraints are not satisfied, start a new truck route with the current request
            if current_truck_route:
                truck_routes.append((current_truck_route, current_delivery_day))  # Append current truck route
            current_truck_route = [[request_id]]  # Start a new truck route
            current_truck_weight = request_weight
            current_truck_distance = depot_to_location_distance
            current_delivery_day = start_day
    
    # Append the last truck route if it's not empty
    if current_truck_route:
        truck_routes.append((current_truck_route, current_delivery_day))
    
    return truck_routes

# Calculate truck routes with constraints
truck_routes_with_constraints = get_truck_routes_with_constraints(
    locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, requests, machines, feasibility_dict
)
print(truck_routes_with_constraints)



[([[1]], 2), ([[2]], 2), ([[3]], 2), ([[4, 5]], 3), ([[6]], 3), ([[7]], 3), ([[8]], 3), ([[9]], 4), ([[10, 11]], 4), ([[12]], 4), ([[13]], 5), ([[14, 15]], 6), ([[16]], 7), ([[17, 18]], 7), ([[19]], 7), ([[20]], 8), ([[21, 22]], 8), ([[23]], 9), ([[24]], 9), ([[25]], 9), ([[26]], 9), ([[27, 28]], 10), ([[29]], 10), ([[30]], 11)]
[([[1]], 2), ([[2]], 2), ([[3]], 2), ([[4]], 3), ([[5]], 3), ([[6]], 3), ([[7]], 3), ([[8]], 3), ([[9]], 4), ([[10]], 4), ([[11]], 4), ([[12]], 4), ([[13]], 5), ([[14]], 6), ([[15]], 6), ([[16]], 7), ([[17, 18]], 7), ([[19]], 7), ([[20]], 8), ([[21]], 8), ([[22]], 8), ([[23]], 9), ([[24]], 9), ([[25]], 9), ([[26]], 9), ([[27]], 10), ([[28]], 10), ([[29]], 10), ([[30]], 11)]


In [1044]:
def constrained_truck_routes_with_depot(truck_routes_with_constraints, depot_loc_ID):
    truck_routes_with_depot = []
    for truck_route, delivery_day in truck_routes_with_constraints:
        # Add depot at the beginning and end of the truck route
        truck_route_with_depot = [depot_loc_ID] + truck_route + [depot_loc_ID]
        truck_routes_with_depot.append((truck_route_with_depot, delivery_day))
    return truck_routes_with_depot

# Calculate truck routes with constraints
truck_routes_with_constraints = get_truck_routes_with_constraints(
    locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, requests, machines, feasibility_dict
)

# Add depot to truck routes
truck_routes_with_depot = constrained_truck_routes_with_depot(truck_routes_with_constraints, depot_loc_ID)

print(truck_routes_with_depot)


[([1, [1], 1], 2), ([1, [2], 1], 2), ([1, [3], 1], 2), ([1, [4], 1], 3), ([1, [5], 1], 3), ([1, [6], 1], 3), ([1, [7], 1], 3), ([1, [8], 1], 3), ([1, [9], 1], 4), ([1, [10], 1], 4), ([1, [11], 1], 4), ([1, [12], 1], 4), ([1, [13], 1], 5), ([1, [14], 1], 6), ([1, [15], 1], 6), ([1, [16], 1], 7), ([1, [17, 18], 1], 7), ([1, [19], 1], 7), ([1, [20], 1], 8), ([1, [21], 1], 8), ([1, [22], 1], 8), ([1, [23], 1], 9), ([1, [24], 1], 9), ([1, [25], 1], 9), ([1, [26], 1], 9), ([1, [27], 1], 10), ([1, [28], 1], 10), ([1, [29], 1], 10), ([1, [30], 1], 11)]


In [1045]:
def extract_truck_routes(truck_routes_with_depot):
    extracted_routes = []
    for route, delivery_day in truck_routes_with_depot:
        request_ids = []
        for segment in route:
            if isinstance(segment, list):  # If segment is a list, extract request IDs from it
                request_ids.extend(segment)
        extracted_routes.append((request_ids, delivery_day))
    return extracted_routes


# Example usage
truck_routes_extracted = extract_truck_routes(truck_routes_with_depot)

def convert_truck_routes_to_locations(truck_routes_with_depot, requests):
    truck_routes_locations = []
    for route, delivery_day in truck_routes_with_depot:
        location_route = [1]  # Start at location ID 1
        for segment in route:
            if isinstance(segment, list):  # Check if segment is a list (request IDs)
                for request_id in segment:
                    location_id = requests[request_id - 1][1]  # Location ID of the request
                    location_route.append(location_id)
        location_route.append(1)  # End at location ID 1
        truck_routes_locations.append((location_route, delivery_day))
    return truck_routes_locations

# Convert truck routes with depot to locations
truck_routes_locations = convert_truck_routes_to_locations(truck_routes_with_depot, requests)

def calculate_truck_distances(locations, truck_routes_locations):
    truck_distances_per_day = {}
    total_truck_distance = 0
    
    for idx, (route, _) in enumerate(truck_routes_locations, start=1):
        total_distance = 0
        for i in range(len(route) - 1):
            loc1 = locations[route[i] - 1]
            loc2 = locations[route[i + 1] - 1]
            distance = ((loc1[1] - loc2[1]) ** 2 + (loc1[2] - loc2[2]) ** 2) ** 0.5  # Euclidean distance
            total_distance += distance
        total_distance = round(total_distance, 2)  # Round to 2 decimal places
        truck_distances_per_day[idx] = total_distance
        total_truck_distance += total_distance
    
    total_truck_distance = round(total_truck_distance, 2)  # Round total distance
    return truck_distances_per_day, total_truck_distance

# Calculate truck distances
truck_distances_per_day, total_truck_distance = calculate_truck_distances(locations, truck_routes_locations)

# Print distances for each truck per day
print("Truck Distances Per Day:")
for truck_idx, distance in truck_distances_per_day.items():
    print(f"Truck {truck_idx}: {distance:.2f}")

# Print total truck distance
print(f"\nTotal Truck Distance: {total_truck_distance:.2f}")


## weet niet of deze nog klopt na aanpassing delivery/installation days
def calculate_idle_days(requests, delivery_days):
    """
    Calculate the idle days for each machine based on delivery and installation days.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, delivery_start_day, delivery_end_day, machine_kind_id, num_requested_machines]
    - installations: List of installations where each installation is represented as a list [installation_day, machine_kind_id]

    Returns:
    - idle_days: Dictionary where keys are machine kind IDs and values are lists of idle days.
    """
    idle_days = {}
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]  # Delivery day is at index 2 of each request
        machine_kind_id = requests[request_id - 1][4]  # Machine kind ID is at index 4
        idle_days[machine_kind_id] = max(0, installation_day - delivery_day)
    return idle_days

# Calculate idle days
idle_days = calculate_idle_days(requests, delivery_days)





# Function to calculate penalties for idle days
def calculate_penalties(delivery_days, requests):
    penalties = {}
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]
        machine_kind_id = requests[request_id - 1][4]
        idle_days = max(0, installation_day - delivery_day - 1)  # Subtract 1 for the day of delivery
        if machine_kind_id not in penalties:
            penalties[machine_kind_id] = 0
        penalties[machine_kind_id] += idle_days
    return penalties


# Function to implement constraints regarding maximum consecutive working days and total distance per day
def apply_constraints(technician_assignments):
    # Implement constraints here
    pass



Truck Distances Per Day:
Truck 1: 612.37
Truck 2: 543.19
Truck 3: 680.50
Truck 4: 680.50
Truck 5: 495.55
Truck 6: 495.55
Truck 7: 543.19
Truck 8: 495.55
Truck 9: 528.95
Truck 10: 674.06
Truck 11: 680.50
Truck 12: 528.95
Truck 13: 495.55
Truck 14: 495.55
Truck 15: 528.95
Truck 16: 674.06
Truck 17: 674.06
Truck 18: 528.95
Truck 19: 680.50
Truck 20: 612.37
Truck 21: 543.19
Truck 22: 674.06
Truck 23: 528.95
Truck 24: 543.19
Truck 25: 495.55
Truck 26: 680.50
Truck 27: 495.55
Truck 28: 528.95
Truck 29: 680.50

Total Truck Distance: 16819.29


In [1046]:
def check_truck_distances(truck_distances_per_day, TRUCK_MAX_DISTANCE):
    exceeded_trucks = []
    for truck_idx, distance in truck_distances_per_day.items():
        if distance > TRUCK_MAX_DISTANCE:
            exceeded_trucks.append(truck_idx)
    return exceeded_trucks

# Call this method after calculating truck distances
exceeded_trucks = check_truck_distances(truck_distances_per_day, TRUCK_MAX_DISTANCE)

if exceeded_trucks:
    print("The following trucks exceeded the maximum distance:")
    for truck_idx in exceeded_trucks:
        print(f"Truck {truck_idx}")
else:
    print("All trucks are within the maximum distance.")


All trucks are within the maximum distance.


In [1047]:
def get_truck_routes_with_depot(locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, requests, machines):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = [depot_loc_ID]  # Start at the depot
    current_truck_weight = 0
    current_truck_distance = 0
    current_delivery_day = None
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        # Calculate the distance from the last location in the route to the current location
        if len(current_truck_route) > 1:
            loc1 = locations[current_truck_route[-1] - 1]
            loc2 = locations[location_id - 1]
            distance = ((loc1[1] - loc2[1]) ** 2 + (loc1[2] - loc2[2]) ** 2) ** 0.5  # Euclidean distance
            current_truck_distance += distance
        
        if current_truck_weight + request_weight <= TRUCK_CAPACITY and start_day == current_delivery_day and current_truck_distance <= TRUCK_MAX_DISTANCE:
            # Check if adding the current request exceeds the truck capacity and if delivery day matches
            current_truck_route.append(location_id)  # Add the location ID to the route
            current_truck_weight += request_weight
        else:
            # End the current route and start a new one
            truck_routes.append((current_truck_route, current_delivery_day))
            current_truck_route = [depot_loc_ID, location_id]  # Start a new truck route
            current_truck_weight = request_weight
            current_delivery_day = start_day
            current_truck_distance = 0
    
    # Append the last truck route if it's not empty
    if current_truck_route:
        truck_routes.append((current_truck_route, current_delivery_day))
    
    return truck_routes

def calculate_truck_distance(locations, route):
    total_distance = 0
    for i in range(len(route) - 1):
        loc1 = locations[route[i] - 1]
        loc2 = locations[route[i + 1] - 1]
        distance = ((loc1[1] - loc2[1]) ** 2 + (loc1[2] - loc2[2]) ** 2) ** 0.5  # Euclidean distance
        total_distance += distance
    return round(total_distance, 2)

# Recalculate truck routes with depot considering maximum distance
truck_routes_with_depot = get_truck_routes_with_depot(locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, requests, machines)


In [1048]:
def get_location_for_request(request):
    return requests[request-1][1]

def nearest_neighbor(point, unvisited_requests, locations):

    min_distance = float('inf')
    nearest_point = None
    point_coordinates = get_location_coordinates(point, locations)
    
    for candidate_request in unvisited_requests:
        request_location_id = get_location_for_request(candidate_request)
        if request_location_id != point:
            candidate_coordinates = get_location_coordinates(request_location_id, locations)
            dist = calculate_distance(point_coordinates, candidate_coordinates)
            if dist is not None and dist < min_distance:
                min_distance = dist
                nearest_point = request_location_id
    return nearest_point


def get_requests_for_location(requests, location_id):
    #Get the list of request IDs corresponding to the given location ID.

    request_ids = [req[0] for req in requests if req[1] == location_id]
    return request_ids
for location in locations:
    location_id = location[0]
    location_request  = [get_requests_for_location(requests, location_id)]
    print(location_request)
    

technician_details = {}

# Define function to print technician details
def print_technician_details(technicians):
    global technician_details
    technician_details.clear()  # Clear existing data
    for technician in technicians:
        tech_id = technician[0]
        max_distance = technician[2]
        max_installations = technician[3]
        technician_details[tech_id] = {'max_distance': max_distance, 'max_installations': max_installations}


# Call function to print technician details
print_technician_details(technicians)

# Now you can print technician_details
print(technician_details)

# Initialize max_distance and max_installations dictionaries
max_distance = {tech_id: details['max_distance'] for tech_id, details in technician_details.items()}
max_installations = {tech_id: details['max_installations'] for tech_id, details in technician_details.items()}



def initialize_home_locations(technicians):
    global home_location_technician
    home_location_technician = {tech[0]: tech[1] for tech in technicians}
    
    # Print the home locations
    print("Home Locations for Technicians:")
    for tech_id, home_location in home_location_technician.items():
        print(f"Technician {tech_id}: Location {home_location}")
    
    return home_location_technician
initialize_home_locations(technicians)
print(home_location_technician)


def nearest_neighbor_for_technician_home(technicians, unvisited_requests, locations):
    home_location_technician = initialize_home_locations(technicians)
    
    for technician_id, home_location in home_location_technician.items():
        nearest_location = nearest_neighbor(home_location, unvisited_requests, locations)
        print(f"The closest location for technician {technician_id} is location {nearest_location}.")
        
        
def transform_skill_set(technicians, technician_id):
    for technician in technicians:
        if technician[0] == technician_id:
            skill_set = technician[4:]  # Get the skill set of the technician (from index 4 onwards)
            transformed_skills = []
            
            # Iterate through the skill_set with index
            for index, value in enumerate(skill_set):
                # If the value is non-zero, append the 1-based index
                if value != 0:
                    transformed_skills.append(index + 1)
                else:
                    # If the value is zero, append zero
                    transformed_skills.append(0)
            
            return transformed_skills
    return None 

def doable_request_skillbased(technicians, requests):
    doable_requests = {}  # Dictionary to store which technician can handle which requests

    for technician in technicians:
        technician_id = technician[0]
        skill_set = transform_skill_set(technicians, technician_id)  # Get the transformed skill set of the technician
        
        for request in requests:
            request_id = request[0]
            machine_type = request[4]  # Fifth entry denotes the machine type

            # Check if the technician has this machine type in their skill set
            if machine_type in skill_set:
                if technician_id not in doable_requests:
                    doable_requests[technician_id] = []
                doable_requests[technician_id].append(request_id)
    
    return doable_requests


doable_requests = doable_request_skillbased(technicians, requests)
doable_requests_copy = doable_request_skillbased(technicians, requests)
for technician_id, request_ids in doable_requests.items():
    print(f"Technician {technician_id} can handle requests: {request_ids}")
transformed_skills = transform_skill_set(technicians, technician_id)
if transformed_skills is not None:
    print(f"Technician {technician_id}: {transformed_skills}")
else:
    print(f"Technician {technician_id} not found.")

    
def check_location_requests_doable(technicians, requests, locations):
    # Dictionary to store the result for each location
    location_results = {}
    
    for location in locations:
        location_id = location[0]
        location_requests = [request for request in requests if request[1] == location_id]
        location_doable_requests = {}
        
        for technician in technicians:
            technician_id = technician[0]
            skill_set = transform_skill_set(technicians, technician_id)  # Get the transformed skill set of the technician
            
            doable_requests = []
            for request in location_requests:
                machine_type = request[4]  # Fifth entry denotes the machine type
                
                # Check if the technician has this machine type in their skill set
                if machine_type in skill_set:
                    doable_requests.append(request[0])
            
            if doable_requests:
                location_doable_requests[technician_id] = doable_requests
        
        location_results[location_id] = location_doable_requests
    
    return location_results

location_results = check_location_requests_doable(technicians, requests, locations)
for location_id, doable_requests in location_results.items():
    print(f"Location {location_id}:")
    for technician_id, request_ids in doable_requests.items():
        print(f"- Technician {technician_id} can handle requests: {request_ids}")

unvisited_requests = [1, 2, 3, 4,5]
current_location = 2

nearest = nearest_neighbor(current_location, unvisited_requests, locations)
print(f"The closest location for {current_location} is location {nearest}.")

# Call the new method
nearest_neighbor_for_technician_home(technicians, unvisited_requests, locations)




[[]]
[[3, 4, 11, 20, 27, 30]]
[[1, 21]]
[[2, 7, 22, 25]]
[[10, 16, 17, 18, 23]]
[[9, 12, 15, 19, 24, 29]]
[[5, 6, 8, 13, 14, 26, 28]]
[[]]
[[]]
{1: {'max_distance': 878, 'max_installations': 8}, 2: {'max_distance': 463, 'max_installations': 5}, 3: {'max_distance': 266, 'max_installations': 5}, 4: {'max_distance': 934, 'max_installations': 6}, 5: {'max_distance': 956, 'max_installations': 9}, 6: {'max_distance': 189, 'max_installations': 8}, 7: {'max_distance': 669, 'max_installations': 9}, 8: {'max_distance': 572, 'max_installations': 9}, 9: {'max_distance': 305, 'max_installations': 8}, 10: {'max_distance': 936, 'max_installations': 6}}
Home Locations for Technicians:
Technician 1: Location 3
Technician 2: Location 8
Technician 3: Location 9
Technician 4: Location 1
Technician 5: Location 1
Technician 6: Location 6
Technician 7: Location 1
Technician 8: Location 1
Technician 9: Location 4
Technician 10: Location 4
{1: 3, 2: 8, 3: 9, 4: 1, 5: 1, 6: 6, 7: 1, 8: 1, 9: 4, 10: 4}
Technicia

In [1049]:
print(doable_requests_copy)

{1: [2, 5, 8, 11, 13, 14, 17, 19, 21, 22, 28, 29], 2: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 3: [2, 5, 8, 11, 13, 14, 17, 19, 21, 22, 28, 29], 4: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 5: [1, 3, 4, 6, 7, 9, 10, 12, 15, 16, 18, 20, 23, 24, 25, 26, 27, 30], 6: [2, 5, 8, 11, 13, 14, 17, 19, 21, 22, 28, 29], 7: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 8: [6, 7, 16, 18, 24, 26, 27, 30], 9: [6, 7, 16, 18, 24, 26, 27, 30], 10: [1, 3, 4, 6, 7, 9, 10, 12, 15, 16, 18, 20, 23, 24, 25, 26, 27, 30]}


In [1050]:
def rare_requests(doable_requests_copy):
    """
    Identify the requests that can be handled by the fewest number of technicians.

    Args:
    - doable_requests: Dictionary where keys are technician IDs and values are lists of request IDs they can handle.

    Returns:
    - rare_requests_dict: Dictionary where keys are request IDs and values are the number of technicians that can handle them.
    """
    # Create a dictionary to count how many technicians can handle each request
    request_count = {}

    # Iterate through the doable_requests dictionary
    for tech_id, requests in doable_requests_copy.items():
        for request_id in requests:
            if request_id in request_count:
                request_count[request_id] += 1
            else:
                request_count[request_id] = 1

    # Debug print to check request counts
    print("Request counts:", request_count)

    # If request_count is empty, return an empty dictionary
    if not request_count:
        return {}

    # Find the minimum count
    min_count = min(request_count.values())

    # Debug print to check the minimum count
    print("Minimum count:", min_count)

    # Identify the requests that have this minimum count
    rare_requests_dict = {req_id: count for req_id, count in request_count.items() if count == min_count}

    return rare_requests_dict


rare_requests_dict = rare_requests(doable_requests_copy)

print("Rare Requests:")
for request_id, count in rare_requests_dict.items():
    print(f"Request {request_id} can be handled by {count} technician(s)")


Request counts: {2: 6, 5: 6, 8: 6, 11: 6, 13: 6, 14: 6, 17: 6, 19: 6, 21: 6, 22: 6, 28: 6, 29: 6, 1: 5, 3: 5, 4: 5, 6: 7, 7: 7, 9: 5, 10: 5, 12: 5, 15: 5, 16: 7, 18: 7, 20: 5, 23: 5, 24: 7, 25: 5, 26: 7, 27: 7, 30: 7}
Minimum count: 5
Rare Requests:
Request 1 can be handled by 5 technician(s)
Request 3 can be handled by 5 technician(s)
Request 4 can be handled by 5 technician(s)
Request 9 can be handled by 5 technician(s)
Request 10 can be handled by 5 technician(s)
Request 12 can be handled by 5 technician(s)
Request 15 can be handled by 5 technician(s)
Request 20 can be handled by 5 technician(s)
Request 23 can be handled by 5 technician(s)
Request 25 can be handled by 5 technician(s)


In [1051]:
def extract_machine_count(requests):
    machine_count = {}
    for req in requests:
        machine_count[req[0]] = req[5]
    return machine_count


machine_count = extract_machine_count(requests)

# Print machine count for each request
print("Machine Count for Each Request:")
for request_id, count in machine_count.items():
    print(f"Request {request_id}: {count} machines")



Machine Count for Each Request:
Request 1: 1 machines
Request 2: 2 machines
Request 3: 2 machines
Request 4: 1 machines
Request 5: 1 machines
Request 6: 2 machines
Request 7: 2 machines
Request 8: 2 machines
Request 9: 2 machines
Request 10: 1 machines
Request 11: 1 machines
Request 12: 2 machines
Request 13: 2 machines
Request 14: 1 machines
Request 15: 1 machines
Request 16: 2 machines
Request 17: 1 machines
Request 18: 1 machines
Request 19: 1 machines
Request 20: 2 machines
Request 21: 1 machines
Request 22: 1 machines
Request 23: 2 machines
Request 24: 2 machines
Request 25: 1 machines
Request 26: 2 machines
Request 27: 1 machines
Request 28: 1 machines
Request 29: 2 machines
Request 30: 2 machines


In [1052]:
def extract_delivery_schedule(truck_routes):
    delivery_schedule = {}
    for route in truck_routes:
        requests, delivery_day = route
        for req in requests:
            for request_id in req:
                delivery_schedule[request_id] = delivery_day
    return delivery_schedule

def print_delivery_schedule(delivery_schedule):
    print("Delivery Schedule:")
    for request_id, delivery_day in sorted(delivery_schedule.items()):
        print(f"Request {request_id}: Delivery Day {delivery_day}")


delivery_schedule = extract_delivery_schedule(truck_routes)

# Print delivery schedule for all requests
print_delivery_schedule(delivery_schedule)

# Determine the number of requests
num_requests = len(delivery_schedule)
print("Number of Requests:", num_requests)

# example to acces delivery day for 1 specific request
print("Delivery Day of Request 1:", delivery_schedule.get(1))  # Example for request 1

def extract_assignment_days(truck_routes):
    assignment_schedule = {}
    for route in truck_routes:
        requests, delivery_day = route
        for req in requests:
            for request_id in req:
                assignment_schedule[request_id] = delivery_day + 1
    return assignment_schedule

# Example usage:
assignment_schedule = extract_assignment_days(truck_routes)

# Print assignment days for all requests
print("Assignment Days:")
for request_id, assignment_day in sorted(assignment_schedule.items()):
    print(f"Request {request_id}: Assignment Day {assignment_day}")

# Example to access assignment day for a specific request
print("Assignment Day of Request 1:", assignment_schedule.get(1))  # Example for request 1





Delivery Schedule:
Request 1: Delivery Day 2
Request 2: Delivery Day 2
Request 3: Delivery Day 2
Request 4: Delivery Day 3
Request 5: Delivery Day 3
Request 6: Delivery Day 3
Request 7: Delivery Day 3
Request 8: Delivery Day 3
Request 9: Delivery Day 4
Request 10: Delivery Day 4
Request 11: Delivery Day 4
Request 12: Delivery Day 4
Request 13: Delivery Day 5
Request 14: Delivery Day 6
Request 15: Delivery Day 6
Request 16: Delivery Day 7
Request 17: Delivery Day 7
Request 18: Delivery Day 7
Request 19: Delivery Day 7
Request 20: Delivery Day 8
Request 21: Delivery Day 8
Request 22: Delivery Day 8
Request 23: Delivery Day 9
Request 24: Delivery Day 9
Request 25: Delivery Day 9
Request 26: Delivery Day 9
Request 27: Delivery Day 10
Request 28: Delivery Day 10
Request 29: Delivery Day 10
Request 30: Delivery Day 11
Number of Requests: 30
Delivery Day of Request 1: 2
Assignment Days:
Request 1: Assignment Day 3
Request 2: Assignment Day 3
Request 3: Assignment Day 3
Request 4: Assignment D

In [1053]:
print(assignment_schedule)

{1: 3, 2: 3, 3: 3, 4: 4, 5: 4, 6: 4, 7: 4, 8: 4, 9: 5, 10: 5, 11: 5, 12: 5, 13: 6, 14: 7, 15: 7, 16: 8, 17: 8, 18: 8, 19: 8, 20: 9, 21: 9, 22: 9, 23: 10, 24: 10, 25: 10, 26: 10, 27: 11, 28: 11, 29: 11, 30: 12}


In [1054]:
def doable_request_skillbased(technicians, requests):
    doable_requests = {}  # Dictionary to store which technician can handle which requests

    for technician in technicians:
        technician_id = technician[0]
        skill_set = transform_skill_set(technicians, technician_id)  # Get the transformed skill set of the technician
        
        for request in requests:
            request_id = request[0]
            machine_type = request[4]  # Fifth entry denotes the machine type

            # Check if the technician has this machine type in their skill set
            if machine_type in skill_set:
                if technician_id not in doable_requests:
                    doable_requests[technician_id] = []
                doable_requests[technician_id].append(request_id)
    
    return doable_requests


doable_requests = doable_request_skillbased(technicians, requests)

print(doable_requests[1])

for technician_id, request_ids in doable_requests.items():
    print(f"Technician {technician_id} can handle requests: {request_ids}")
    transformed_skills = transform_skill_set(technicians, technician_id)
    if transformed_skills is not None:
        print(f"Technician {technician_id}: skill set {transformed_skills}")
    else:
        print(f"Technician {technician_id} not found.")

    


[2, 5, 8, 11, 13, 14, 17, 19, 21, 22, 28, 29]
Technician 1 can handle requests: [2, 5, 8, 11, 13, 14, 17, 19, 21, 22, 28, 29]
Technician 1: skill set [0, 0, 3]
Technician 2 can handle requests: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Technician 2: skill set [1, 2, 3]
Technician 3 can handle requests: [2, 5, 8, 11, 13, 14, 17, 19, 21, 22, 28, 29]
Technician 3: skill set [0, 0, 3]
Technician 4 can handle requests: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Technician 4: skill set [1, 2, 3]
Technician 5 can handle requests: [1, 3, 4, 6, 7, 9, 10, 12, 15, 16, 18, 20, 23, 24, 25, 26, 27, 30]
Technician 5: skill set [1, 2, 0]
Technician 6 can handle requests: [2, 5, 8, 11, 13, 14, 17, 19, 21, 22, 28, 29]
Technician 6: skill set [0, 0, 3]
Technician 7 can handle requests: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,

In [1055]:
def home_location_technician_coordinates(home_location_technician, locations):
    """
    Get the coordinates of technicians' home locations.

    Args:
    - home_location_technician: Dictionary where keys are technician IDs and values are their home location IDs.
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - home_coordinates: Dictionary where keys are technician IDs and values are tuples (x, y) representing their home location coordinates.
    """
    home_coordinates = {}
    for tech_id, location_id in home_location_technician.items():
        coordinates = get_location_coordinates(location_id, locations)
        if coordinates:
            home_coordinates[tech_id] = coordinates
    return home_coordinates

home_coordinates = home_location_technician_coordinates(home_location_technician, locations)
print("Technicians' home coordinates:")
for tech_id, coordinates in home_coordinates.items():
    print(f"Technician {tech_id}: {coordinates}")

Technicians' home coordinates:
Technician 1: (274, 532)
Technician 2: (234, 340)
Technician 3: (206, 343)
Technician 4: (356, 237)
Technician 5: (356, 237)
Technician 6: (541, 426)
Technician 7: (356, 237)
Technician 8: (356, 237)
Technician 9: (533, 31)
Technician 10: (533, 31)


In [1056]:

def request_location_coordinates(requests, locations):
    """
    Get the coordinates of each request location.

    Args:
    - requests: List of requests where each request is represented as [request_id, _, location_id, _, _, _].
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - request_coordinates: Dictionary where keys are request IDs and values are tuples (x, y) representing their location coordinates.
    """
    request_coordinates = {}
    for request in requests:
        request_id, location_id, *_ = request
        coordinates = get_location_coordinates(location_id, locations)
        if coordinates:
            request_coordinates[request_id] = coordinates
    return request_coordinates

request_coordinates = request_location_coordinates(requests, locations)
print("Requests' location coordinates:")
for request_id, coordinates in request_coordinates.items():
    print(f"Request {request_id}: {coordinates}")

    
def get_request_location_ids(requests):
    """
    Extract and print the location IDs of each request.

    Args:
    - requests: List of lists where each inner list represents a request.

    Returns:
    - location_ids: A list of location IDs extracted from the requests.
    """
    location_ids = [request[1] for request in requests]
    return location_ids


# Get all request location IDs
location_ids = get_request_location_ids(requests)

# Print all request location IDs
for i, location_id in enumerate(location_ids, start=1):
    print(f"Request ID {i}: Location ID {location_id}")


Requests' location coordinates:
Request 1: (274, 532)
Request 2: (533, 31)
Request 3: (369, 577)
Request 4: (369, 577)
Request 5: (464, 460)
Request 6: (464, 460)
Request 7: (533, 31)
Request 8: (464, 460)
Request 9: (541, 426)
Request 10: (298, 569)
Request 11: (369, 577)
Request 12: (541, 426)
Request 13: (464, 460)
Request 14: (464, 460)
Request 15: (541, 426)
Request 16: (298, 569)
Request 17: (298, 569)
Request 18: (298, 569)
Request 19: (541, 426)
Request 20: (369, 577)
Request 21: (274, 532)
Request 22: (533, 31)
Request 23: (298, 569)
Request 24: (541, 426)
Request 25: (533, 31)
Request 26: (464, 460)
Request 27: (369, 577)
Request 28: (464, 460)
Request 29: (541, 426)
Request 30: (369, 577)
Request ID 1: Location ID 3
Request ID 2: Location ID 4
Request ID 3: Location ID 2
Request ID 4: Location ID 2
Request ID 5: Location ID 7
Request ID 6: Location ID 7
Request ID 7: Location ID 4
Request ID 8: Location ID 7
Request ID 9: Location ID 6
Request ID 10: Location ID 5
Request ID

In [1057]:
def distance_between_home_and_request(home_coordinates, request_coordinates, home_id, request_id):
    """
    Calculate the distance between a home location and a request location.

    Args:
    - home_coordinates: Dictionary containing home IDs as keys and their coordinates as values.
    - request_coordinates: Dictionary containing request IDs as keys and their coordinates as values.
    - home_id: ID of the home.
    - request_id: ID of the request.

    Returns:
    - distance: The Euclidean distance between the home and request locations.
    """
    home_location = home_coordinates.get(home_id)
    request_location = request_coordinates.get(request_id)
    if home_location is None:
        raise ValueError(f"Home ID {home_id} not found in home_coordinates.")
    if request_location is None:
        raise ValueError(f"Request ID {request_id} not found in request_coordinates.")
    
    return calculate_distance(home_location, request_location)

def print_distances_for_all_technicians_and_requests(home_coordinates, request_coordinates):
    """
    Print distances between each technician's home and each request.

    Args:
    - home_coordinates: Dictionary containing home IDs as keys and their coordinates as values.
    - request_coordinates: Dictionary containing request IDs as keys and their coordinates as values.
    """
    for home_id, home_location in home_coordinates.items():
        for request_id, request_location in request_coordinates.items():
            distance = distance_between_home_and_request(home_coordinates, request_coordinates, home_id, request_id)
            print(f"Distance between home {home_id} and request {request_id}: {distance}")

print_distances_for_all_technicians_and_requests(home_coordinates, request_coordinates)

Distance between home 1 and request 1: 0
Distance between home 1 and request 2: 564
Distance between home 1 and request 3: 106
Distance between home 1 and request 4: 106
Distance between home 1 and request 5: 204
Distance between home 1 and request 6: 204
Distance between home 1 and request 7: 564
Distance between home 1 and request 8: 204
Distance between home 1 and request 9: 288
Distance between home 1 and request 10: 45
Distance between home 1 and request 11: 106
Distance between home 1 and request 12: 288
Distance between home 1 and request 13: 204
Distance between home 1 and request 14: 204
Distance between home 1 and request 15: 288
Distance between home 1 and request 16: 45
Distance between home 1 and request 17: 45
Distance between home 1 and request 18: 45
Distance between home 1 and request 19: 288
Distance between home 1 and request 20: 106
Distance between home 1 and request 21: 0
Distance between home 1 and request 22: 564
Distance between home 1 and request 23: 45
Distan

In [1058]:
def calculate_distances_for_all_technicians_and_requests(home_coordinates, request_coordinates):
    """
    Calculate and store distances between each technician's home and each request.

    Args:
    - home_coordinates: Dictionary containing home IDs as keys and their coordinates as values.
    - request_coordinates: Dictionary containing request IDs as keys and their coordinates as values.

    Returns:
    - distances: Dictionary with keys as (home_id, request_id) tuples and values as distances.
    """
    distances = {}
    for home_id, home_location in home_coordinates.items():
        for request_id, request_location in request_coordinates.items():
            distance = distance_between_home_and_request(home_coordinates, request_coordinates, home_id, request_id)
            distances[(home_id, request_id)] = distance
            print(f"Distance between home {home_id} and request {request_id}: {distance}")
    return distances


distances = calculate_distances_for_all_technicians_and_requests(home_coordinates, request_coordinates)



Distance between home 1 and request 1: 0
Distance between home 1 and request 2: 564
Distance between home 1 and request 3: 106
Distance between home 1 and request 4: 106
Distance between home 1 and request 5: 204
Distance between home 1 and request 6: 204
Distance between home 1 and request 7: 564
Distance between home 1 and request 8: 204
Distance between home 1 and request 9: 288
Distance between home 1 and request 10: 45
Distance between home 1 and request 11: 106
Distance between home 1 and request 12: 288
Distance between home 1 and request 13: 204
Distance between home 1 and request 14: 204
Distance between home 1 and request 15: 288
Distance between home 1 and request 16: 45
Distance between home 1 and request 17: 45
Distance between home 1 and request 18: 45
Distance between home 1 and request 19: 288
Distance between home 1 and request 20: 106
Distance between home 1 and request 21: 0
Distance between home 1 and request 22: 564
Distance between home 1 and request 23: 45
Distan

In [1059]:
print(home_location_technician)
print(home_coordinates)
print(home_coordinates.get(1))
print(request_coordinates)
distances[(1,9)]


{1: 3, 2: 8, 3: 9, 4: 1, 5: 1, 6: 6, 7: 1, 8: 1, 9: 4, 10: 4}
{1: (274, 532), 2: (234, 340), 3: (206, 343), 4: (356, 237), 5: (356, 237), 6: (541, 426), 7: (356, 237), 8: (356, 237), 9: (533, 31), 10: (533, 31)}
(274, 532)
{1: (274, 532), 2: (533, 31), 3: (369, 577), 4: (369, 577), 5: (464, 460), 6: (464, 460), 7: (533, 31), 8: (464, 460), 9: (541, 426), 10: (298, 569), 11: (369, 577), 12: (541, 426), 13: (464, 460), 14: (464, 460), 15: (541, 426), 16: (298, 569), 17: (298, 569), 18: (298, 569), 19: (541, 426), 20: (369, 577), 21: (274, 532), 22: (533, 31), 23: (298, 569), 24: (541, 426), 25: (533, 31), 26: (464, 460), 27: (369, 577), 28: (464, 460), 29: (541, 426), 30: (369, 577)}


288

In [1060]:
def check_max_distance_exceeded(distances, max_distance):
    """
    Check if the round trip distance for each technician exceeds their maximum distance.

    Args:
    - distances: Dictionary containing distances between each technician's home and each request.
    - max_distance: Dictionary containing maximum distances for each technician.

    Returns:
    - results: Dictionary containing tuples of (home_id, request_id) as keys and boolean values indicating
               whether the round trip distance exceeds the maximum distance for the corresponding technician.
    """
    results = {}
    for (home_id, request_id), distance in distances.items():
        round_trip_distance = 2 * distance
        max_dist = max_distance.get(home_id, float('inf'))
        exceeds_max_distance = round_trip_distance > max_dist
        results[(home_id, request_id)] = exceeds_max_distance
    return results

# Assuming distances is already calculated from the previous code
exceeded_distances = check_max_distance_exceeded(distances, max_distance)

# Print the results
for (home_id, request_id), exceeds in exceeded_distances.items():
    status = "exceeds" if exceeds else "does not exceed"
    print(f"Round trip distance for home {home_id} to request {request_id} {'exceeds' if exceeds else 'does not exceed'} the max distance.")


Round trip distance for home 1 to request 1 does not exceed the max distance.
Round trip distance for home 1 to request 2 exceeds the max distance.
Round trip distance for home 1 to request 3 does not exceed the max distance.
Round trip distance for home 1 to request 4 does not exceed the max distance.
Round trip distance for home 1 to request 5 does not exceed the max distance.
Round trip distance for home 1 to request 6 does not exceed the max distance.
Round trip distance for home 1 to request 7 exceeds the max distance.
Round trip distance for home 1 to request 8 does not exceed the max distance.
Round trip distance for home 1 to request 9 does not exceed the max distance.
Round trip distance for home 1 to request 10 does not exceed the max distance.
Round trip distance for home 1 to request 11 does not exceed the max distance.
Round trip distance for home 1 to request 12 does not exceed the max distance.
Round trip distance for home 1 to request 13 does not exceed the max distance

In [1061]:
exceeded_distances

{(1, 1): False,
 (1, 2): True,
 (1, 3): False,
 (1, 4): False,
 (1, 5): False,
 (1, 6): False,
 (1, 7): True,
 (1, 8): False,
 (1, 9): False,
 (1, 10): False,
 (1, 11): False,
 (1, 12): False,
 (1, 13): False,
 (1, 14): False,
 (1, 15): False,
 (1, 16): False,
 (1, 17): False,
 (1, 18): False,
 (1, 19): False,
 (1, 20): False,
 (1, 21): False,
 (1, 22): True,
 (1, 23): False,
 (1, 24): False,
 (1, 25): True,
 (1, 26): False,
 (1, 27): False,
 (1, 28): False,
 (1, 29): False,
 (1, 30): False,
 (2, 1): False,
 (2, 2): True,
 (2, 3): True,
 (2, 4): True,
 (2, 5): True,
 (2, 6): True,
 (2, 7): True,
 (2, 8): True,
 (2, 9): True,
 (2, 10): True,
 (2, 11): True,
 (2, 12): True,
 (2, 13): True,
 (2, 14): True,
 (2, 15): True,
 (2, 16): True,
 (2, 17): True,
 (2, 18): True,
 (2, 19): True,
 (2, 20): True,
 (2, 21): False,
 (2, 22): True,
 (2, 23): True,
 (2, 24): True,
 (2, 25): True,
 (2, 26): True,
 (2, 27): True,
 (2, 28): True,
 (2, 29): True,
 (2, 30): True,
 (3, 1): True,
 (3, 2): True,


In [1062]:
def doable_requests_distance_based(exceeded_distances):
    """
    Check which requests each technician can handle based on the distances and maximum distances.

    Args:
    - exceeded_distances: Dictionary containing tuples of (home_id, request_id) as keys and boolean values indicating
                          whether the round trip distance exceeds the maximum distance for the corresponding technician.

    Returns:
    - doable_requests: Dictionary where keys are technician ids and values are lists of requests that the technician can handle.
    """
    doable_requests = {}
    for (home_id, request_id), exceeds in exceeded_distances.items():
        if not exceeds:
            if home_id not in doable_requests:
                doable_requests[home_id] = []
            doable_requests[home_id].append(request_id)
    return doable_requests

# Get doable requests based on distance
doable_requests_distance_based = doable_requests_distance_based(exceeded_distances)

# Print the result
for technician, requests in doable_requests_distance_based.items():
    print(f"Technician {technician} can handle requests: {requests}")


Technician 1 can handle requests: [1, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 26, 27, 28, 29, 30]
Technician 2 can handle requests: [1, 21]
Technician 4 can handle requests: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Technician 5 can handle requests: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Technician 6 can handle requests: [5, 6, 8, 9, 12, 13, 14, 15, 19, 24, 26, 28, 29]
Technician 7 can handle requests: [1, 2, 5, 6, 7, 8, 9, 12, 13, 14, 15, 19, 21, 22, 24, 25, 26, 28, 29]
Technician 8 can handle requests: [2, 5, 6, 7, 8, 9, 12, 13, 14, 15, 19, 22, 24, 25, 26, 28, 29]
Technician 9 can handle requests: [2, 7, 22, 25]
Technician 10 can handle requests: [2, 5, 6, 7, 8, 9, 12, 13, 14, 15, 19, 22, 24, 25, 26, 28, 29]


In [1063]:
# dit returned een list met requests die de technicain kan doen based op zn skills en op zn distance vanaf home

def doable_requests_double_check(doable_requests_distance_based, doable_requests):
    double_check_requests = {}
    for tech_id in doable_requests_distance_based.keys():
        common_requests = set(doable_requests_distance_based[tech_id]) & set(doable_requests[tech_id])
        double_check_requests[tech_id] = list(common_requests)
    return double_check_requests

double_check_requests = doable_requests_double_check(doable_requests_distance_based, doable_requests)
print(double_check_requests)




{1: [5, 8, 11, 13, 14, 17, 19, 21, 28, 29], 2: [1, 21], 4: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 5: [1, 3, 4, 6, 7, 9, 10, 12, 15, 16, 18, 20, 23, 24, 25, 26, 27, 30], 6: [5, 8, 13, 14, 19, 28, 29], 7: [1, 2, 5, 6, 7, 8, 9, 12, 13, 14, 15, 19, 21, 22, 24, 25, 26, 28, 29], 8: [24, 26, 6, 7], 9: [7], 10: [6, 7, 9, 12, 15, 24, 25, 26]}


In [1064]:
def rare_requests_double_checked(double_check_requests):
    # Count how many technicians can handle each request and keep track of the technicians
    request_count = {}
    request_technicians = {}
    for tech_id, tech_requests in double_check_requests.items():
        for request in tech_requests:
            request_count[request] = request_count.get(request, 0) + 1
            request_technicians.setdefault(request, []).append(tech_id)
    
    # Find the minimum count
    min_count = min(request_count.values()) if request_count else 0
    
    # Identify the requests that have this minimum count
    rare_requests_dict = {req_id: {'count': count, 'technicians': request_technicians[req_id]} for req_id, count in request_count.items() if count == min_count}
    
    return rare_requests_dict

# Example usage:
rare_requests_dict = rare_requests_double_checked(double_check_requests)
print("Rare Requests:")
for request_id, data in rare_requests_dict.items():
    count = data['count']
    technicians_copy = data['technicians']
    print(f"Request {request_id} can be handled by the least number of technicians ({count} technicians): Technicians {technicians_copy}")
print(technicians)
print(rare_requests_dict)

Rare Requests:
Request 11 can be handled by the least number of technicians (2 technicians): Technicians [1, 4]
Request 17 can be handled by the least number of technicians (2 technicians): Technicians [1, 4]
Request 2 can be handled by the least number of technicians (2 technicians): Technicians [4, 7]
Request 3 can be handled by the least number of technicians (2 technicians): Technicians [4, 5]
Request 4 can be handled by the least number of technicians (2 technicians): Technicians [4, 5]
Request 10 can be handled by the least number of technicians (2 technicians): Technicians [4, 5]
Request 16 can be handled by the least number of technicians (2 technicians): Technicians [4, 5]
Request 18 can be handled by the least number of technicians (2 technicians): Technicians [4, 5]
Request 20 can be handled by the least number of technicians (2 technicians): Technicians [4, 5]
Request 22 can be handled by the least number of technicians (2 technicians): Technicians [4, 7]
Request 23 can be 

In [1065]:
def technician_rare_request_count(double_check_requests, rare_requests):
    # Initialize a dictionary to store the rare request count for each technician
    technician_rare_counts = {tech_id: 0 for tech_id in double_check_requests.keys()}
    
    # Iterate through rare requests and update the count for each technician
    for req_id, data in rare_requests.items():
        technicians = data['technicians']
        for tech_id in technicians:
            technician_rare_counts[tech_id] += 1
    
    return technician_rare_counts

# Example usage:
technician_rare_counts = technician_rare_request_count(double_check_requests, rare_requests_dict)
print("Technician Rare Request Counts:")
for tech_id, count in technician_rare_counts.items():
    print(f"Technician {tech_id} has {count} rare requests.")

print(technician_rare_counts)
print(rare_requests)
print(rare_requests_dict)

Technician Rare Request Counts:
Technician 1 has 2 rare requests.
Technician 2 has 0 rare requests.
Technician 4 has 13 rare requests.
Technician 5 has 9 rare requests.
Technician 6 has 0 rare requests.
Technician 7 has 2 rare requests.
Technician 8 has 0 rare requests.
Technician 9 has 0 rare requests.
Technician 10 has 0 rare requests.
{1: 2, 2: 0, 4: 13, 5: 9, 6: 0, 7: 2, 8: 0, 9: 0, 10: 0}
<function rare_requests at 0x0000019D111134C0>
{11: {'count': 2, 'technicians': [1, 4]}, 17: {'count': 2, 'technicians': [1, 4]}, 2: {'count': 2, 'technicians': [4, 7]}, 3: {'count': 2, 'technicians': [4, 5]}, 4: {'count': 2, 'technicians': [4, 5]}, 10: {'count': 2, 'technicians': [4, 5]}, 16: {'count': 2, 'technicians': [4, 5]}, 18: {'count': 2, 'technicians': [4, 5]}, 20: {'count': 2, 'technicians': [4, 5]}, 22: {'count': 2, 'technicians': [4, 7]}, 23: {'count': 2, 'technicians': [4, 5]}, 27: {'count': 2, 'technicians': [4, 5]}, 30: {'count': 2, 'technicians': [4, 5]}}


In [1066]:
def find_requests_with_same_day_and_loc(assignment_schedule, request_coordinates, location_ids):
    """
    Find requests with the same location ID and assignment day.

    Args:
    - assignment_schedule: Dictionary where keys are request IDs and values are assignment days.
    - request_coordinates: Dictionary where keys are request IDs and values are tuples representing location coordinates.
    - location_ids: List of location IDs extracted from the requests.

    Returns:
    - requests_same_day_loc: Dictionary where keys are tuples (location ID, assignment day) and values are lists of request IDs.
    """
    requests_same_day_loc = {}
    for request_id, assignment_day in assignment_schedule.items():
        location_id = location_ids[request_id - 1]  # Adjusting index since request_id starts from 1
        if location_id is not None:
            key = (location_id, assignment_day)
            requests_same_day_loc.setdefault(key, []).append(request_id)
    return {key: requests for key, requests in requests_same_day_loc.items() if len(requests) > 1}

# Example usage:
requests_same_day_loc = find_requests_with_same_day_and_loc(assignment_schedule, request_coordinates, location_ids)

# Print requests with the same location and assignment day
print("Requests with the Same Location ID and Assignment Day:")
for (location_id, assignment_day), requests in requests_same_day_loc.items():
    print(f"Location ID {location_id}, Assignment Day {assignment_day}: Requests {requests}")


Requests with the Same Location ID and Assignment Day:
Location ID 7, Assignment Day 4: Requests [5, 6, 8]
Location ID 6, Assignment Day 5: Requests [9, 12]
Location ID 5, Assignment Day 8: Requests [16, 17, 18]


In [1067]:
requests_same_day_loc

{(7, 4): [5, 6, 8], (6, 5): [9, 12], (5, 8): [16, 17, 18]}

In [1068]:
def find_request_pairs_across_days(assignment_schedule, request_coordinates):
    """
    Find pairs of requests with the same or different assignment days and calculate the distance between them.

    Args:
    - assignment_schedule: Dictionary where keys are request IDs and values are assignment days.
    - request_coordinates: Dictionary where keys are request IDs and values are tuples representing location coordinates.

    Returns:
    - request_pairs_across_days: Dictionary where keys are assignment days and values are lists of tuples ((request ID 1, request ID 2), distance).
    """
    request_pairs_across_days = {}
    max_assignment_day = max(assignment_schedule.values())

    for day in range(1, max_assignment_day + 1):
        requests_to_consider = [req for req, assigned_day in assignment_schedule.items() if assigned_day <= day]

        pairs_with_distance = []
        for req1, req2 in combinations(requests_to_consider, 2):
            coord1 = request_coordinates[req1]
            coord2 = request_coordinates[req2]
            distance = calculate_distance(coord1, coord2)
            pairs_with_distance.append(((req1, req2), distance))
        
        request_pairs_across_days[day] = pairs_with_distance

    return request_pairs_across_days

# Example usage:
request_pairs_across_days = find_request_pairs_across_days(assignment_schedule, request_coordinates)

# Print request pairs considering all assignment days
print("Request Pairs Considering All Assignment Days:")
for day, pairs_distance in request_pairs_across_days.items():
    print(f"Assignment Day {day}:")
    for pair, distance in pairs_distance:
        request_id1, request_id2 = pair
        print(f"Request Pair ({request_id1}, {request_id2}): Distance {distance}")



Request Pairs Considering All Assignment Days:
Assignment Day 1:
Assignment Day 2:
Assignment Day 3:
Request Pair (1, 2): Distance 564
Request Pair (1, 3): Distance 106
Request Pair (2, 3): Distance 571
Assignment Day 4:
Request Pair (1, 2): Distance 564
Request Pair (1, 3): Distance 106
Request Pair (1, 4): Distance 106
Request Pair (1, 5): Distance 204
Request Pair (1, 6): Distance 204
Request Pair (1, 7): Distance 564
Request Pair (1, 8): Distance 204
Request Pair (2, 3): Distance 571
Request Pair (2, 4): Distance 571
Request Pair (2, 5): Distance 435
Request Pair (2, 6): Distance 435
Request Pair (2, 7): Distance 0
Request Pair (2, 8): Distance 435
Request Pair (3, 4): Distance 0
Request Pair (3, 5): Distance 151
Request Pair (3, 6): Distance 151
Request Pair (3, 7): Distance 571
Request Pair (3, 8): Distance 151
Request Pair (4, 5): Distance 151
Request Pair (4, 6): Distance 151
Request Pair (4, 7): Distance 571
Request Pair (4, 8): Distance 151
Request Pair (5, 6): Distance 0
Req

In [1069]:
def calculate_round_trip_distances(home_coordinates, request_pairs_across_days, request_coordinates):
    """
    Calculate the round-trip distance from the technician's home depot to the first request,
    to the second request, and back to the home depot.

    Args:
    - home_coordinates: Dictionary where keys are technician IDs and values are tuples representing the coordinates of the home depots.
    - request_pairs_across_days: Dictionary where keys are assignment days and values are lists of tuples ((request ID 1, request ID 2), distance).
    - request_coordinates: Dictionary where keys are request IDs and values are tuples representing location coordinates.

    Returns:
    - technician_round_trip_distances: Dictionary where keys are technician IDs and values are dictionaries with keys as assignment days
      and values are lists of tuples ((request ID 1, request ID 2), round-trip distance).
    """
    technician_round_trip_distances = {}
    for technician, home_coord in home_coordinates.items():
        round_trip_distances = {}
        for day, pairs_distance in request_pairs_across_days.items():
            day_distances = []
            for pair, distance in pairs_distance:
                request_id1, request_id2 = pair
                coord1 = request_coordinates[request_id1]
                coord2 = request_coordinates[request_id2]
                distance_to_first = calculate_distance(home_coord, coord1)
                distance_from_second = calculate_distance(coord2, home_coord)
                round_trip_distance = distance_to_first + distance + distance_from_second
                day_distances.append((pair, round_trip_distance))
            round_trip_distances[day] = day_distances
        technician_round_trip_distances[technician] = round_trip_distances
    return technician_round_trip_distances

# Example usage:
request_pairs_across_days = find_request_pairs_across_days(assignment_schedule, request_coordinates)
technician_round_trip_distances = calculate_round_trip_distances(home_coordinates, request_pairs_across_days, request_coordinates)

# Print the round-trip distances for each technician
for technician, round_trip_distances in technician_round_trip_distances.items():
    print(f"Technician {technician}:")
    for day, pairs_distance in round_trip_distances.items():
        print(f"  Assignment Day {day}:")
        for pair, distance in pairs_distance:
            request_id1, request_id2 = pair
            print(f"    Request Pair ({request_id1}, {request_id2}): Round-Trip Distance {distance}")

technician_round_trip_distances


Technician 1:
  Assignment Day 1:
  Assignment Day 2:
  Assignment Day 3:
    Request Pair (1, 2): Round-Trip Distance 1128
    Request Pair (1, 3): Round-Trip Distance 212
    Request Pair (2, 3): Round-Trip Distance 1241
  Assignment Day 4:
    Request Pair (1, 2): Round-Trip Distance 1128
    Request Pair (1, 3): Round-Trip Distance 212
    Request Pair (1, 4): Round-Trip Distance 212
    Request Pair (1, 5): Round-Trip Distance 408
    Request Pair (1, 6): Round-Trip Distance 408
    Request Pair (1, 7): Round-Trip Distance 1128
    Request Pair (1, 8): Round-Trip Distance 408
    Request Pair (2, 3): Round-Trip Distance 1241
    Request Pair (2, 4): Round-Trip Distance 1241
    Request Pair (2, 5): Round-Trip Distance 1203
    Request Pair (2, 6): Round-Trip Distance 1203
    Request Pair (2, 7): Round-Trip Distance 1128
    Request Pair (2, 8): Round-Trip Distance 1203
    Request Pair (3, 4): Round-Trip Distance 212
    Request Pair (3, 5): Round-Trip Distance 461
    Request Pa

{1: {1: [],
  2: [],
  3: [((1, 2), 1128), ((1, 3), 212), ((2, 3), 1241)],
  4: [((1, 2), 1128),
   ((1, 3), 212),
   ((1, 4), 212),
   ((1, 5), 408),
   ((1, 6), 408),
   ((1, 7), 1128),
   ((1, 8), 408),
   ((2, 3), 1241),
   ((2, 4), 1241),
   ((2, 5), 1203),
   ((2, 6), 1203),
   ((2, 7), 1128),
   ((2, 8), 1203),
   ((3, 4), 212),
   ((3, 5), 461),
   ((3, 6), 461),
   ((3, 7), 1241),
   ((3, 8), 461),
   ((4, 5), 461),
   ((4, 6), 461),
   ((4, 7), 1241),
   ((4, 8), 461),
   ((5, 6), 408),
   ((5, 7), 1203),
   ((5, 8), 408),
   ((6, 7), 1203),
   ((6, 8), 408),
   ((7, 8), 1203)],
  5: [((1, 2), 1128),
   ((1, 3), 212),
   ((1, 4), 212),
   ((1, 5), 408),
   ((1, 6), 408),
   ((1, 7), 1128),
   ((1, 8), 408),
   ((1, 9), 576),
   ((1, 10), 90),
   ((1, 11), 212),
   ((1, 12), 576),
   ((2, 3), 1241),
   ((2, 4), 1241),
   ((2, 5), 1203),
   ((2, 6), 1203),
   ((2, 7), 1128),
   ((2, 8), 1203),
   ((2, 9), 1248),
   ((2, 10), 1197),
   ((2, 11), 1241),
   ((2, 12), 1248),
   ((3

In [1070]:
def find_lowest_distance_pairs(technician_round_trip_distances):
    """
    Find the two request pairs with the lowest round-trip distances for each technician on each day.

    Args:
    - technician_round_trip_distances: Dictionary where keys are technician IDs and values are dictionaries with keys as assignment days
      and values are lists of tuples ((request ID 1, request ID 2), round-trip distance).

    Returns:
    - lowest_distance_pairs: Dictionary where keys are technician IDs and values are dictionaries with keys as assignment days
      and values are lists of the two tuples ((request ID 1, request ID 2), round-trip distance) with the lowest distances.
    """
    lowest_distance_pairs = {}
    for technician, round_trip_distances in technician_round_trip_distances.items():
        tech_lowest_pairs = {}
        for day, pairs_distance in round_trip_distances.items():
            # Sort the pairs by distance and take the two with the lowest distance
            sorted_pairs = sorted(pairs_distance, key=lambda x: x[1])[:2]
            tech_lowest_pairs[day] = sorted_pairs
        lowest_distance_pairs[technician] = tech_lowest_pairs
    return lowest_distance_pairs

# Calculate the round-trip distances
request_pairs_across_days = find_request_pairs_across_days(assignment_schedule, request_coordinates)
technician_round_trip_distances = calculate_round_trip_distances(home_coordinates, request_pairs_across_days, request_coordinates)

# Find the two pairs with the lowest distances for each technician on each day
lowest_distance_pairs = find_lowest_distance_pairs(technician_round_trip_distances)

# Print the lowest distance pairs for each technician
for technician, lowest_pairs in lowest_distance_pairs.items():
    print(f"Technician {technician}:")
    for day, pairs_distance in lowest_pairs.items():
        print(f"  Assignment Day {day}:")
        for pair, distance in pairs_distance:
            request_id1, request_id2 = pair
            print(f"    Request Pair ({request_id1}, {request_id2}): Round-Trip Distance {distance}")

lowest_distance_pairs



Technician 1:
  Assignment Day 1:
  Assignment Day 2:
  Assignment Day 3:
    Request Pair (1, 3): Round-Trip Distance 212
    Request Pair (1, 2): Round-Trip Distance 1128
  Assignment Day 4:
    Request Pair (1, 3): Round-Trip Distance 212
    Request Pair (1, 4): Round-Trip Distance 212
  Assignment Day 5:
    Request Pair (1, 10): Round-Trip Distance 90
    Request Pair (1, 3): Round-Trip Distance 212
  Assignment Day 6:
    Request Pair (1, 10): Round-Trip Distance 90
    Request Pair (1, 3): Round-Trip Distance 212
  Assignment Day 7:
    Request Pair (1, 10): Round-Trip Distance 90
    Request Pair (1, 3): Round-Trip Distance 212
  Assignment Day 8:
    Request Pair (1, 10): Round-Trip Distance 90
    Request Pair (1, 16): Round-Trip Distance 90
  Assignment Day 9:
    Request Pair (1, 21): Round-Trip Distance 0
    Request Pair (1, 10): Round-Trip Distance 90
  Assignment Day 10:
    Request Pair (1, 21): Round-Trip Distance 0
    Request Pair (1, 10): Round-Trip Distance 90
  

{1: {1: [],
  2: [],
  3: [((1, 3), 212), ((1, 2), 1128)],
  4: [((1, 3), 212), ((1, 4), 212)],
  5: [((1, 10), 90), ((1, 3), 212)],
  6: [((1, 10), 90), ((1, 3), 212)],
  7: [((1, 10), 90), ((1, 3), 212)],
  8: [((1, 10), 90), ((1, 16), 90)],
  9: [((1, 21), 0), ((1, 10), 90)],
  10: [((1, 21), 0), ((1, 10), 90)],
  11: [((1, 21), 0), ((1, 10), 90)],
  12: [((1, 21), 0), ((1, 10), 90)]},
 2: {1: [],
  2: [],
  3: [((1, 3), 576), ((1, 2), 1191)],
  4: [((5, 6), 520), ((5, 8), 520)],
  5: [((1, 10), 480), ((5, 6), 520)],
  6: [((1, 10), 480), ((5, 6), 520)],
  7: [((1, 10), 480), ((5, 6), 520)],
  8: [((10, 16), 476), ((10, 17), 476)],
  9: [((1, 21), 394), ((10, 16), 476)],
  10: [((1, 21), 394), ((10, 16), 476)],
  11: [((1, 21), 394), ((10, 16), 476)],
  12: [((1, 21), 394), ((10, 16), 476)]},
 3: {1: [],
  2: [],
  3: [((1, 3), 593), ((1, 2), 1217)],
  4: [((5, 6), 568), ((5, 8), 568)],
  5: [((1, 10), 491), ((5, 6), 568)],
  6: [((1, 10), 491), ((5, 6), 568)],
  7: [((1, 10), 491),

In [1071]:
def can_handle_request_combinations(technician_round_trip_distances, max_distance):
    """
    Determine if a technician can handle the request combinations within their maximum travel distance.

    Args:
    - technician_round_trip_distances: Dictionary where keys are technician IDs and values are dictionaries with keys as assignment days
      and values are lists of tuples ((request ID 1, request ID 2), round-trip distance).
    - max_distance: Dictionary where keys are technician IDs and values are the maximum travel distance for that technician.

    Returns:
    - feasible_combinations: Dictionary where keys are technician IDs and values are dictionaries with keys as assignment days
      and values are lists of tuples ((request ID 1, request ID 2), round-trip distance) that are within the max distance.
    """
    feasible_combinations = {}
    for technician, round_trip_distances in technician_round_trip_distances.items():
        tech_feasible_pairs = {}
        for day, pairs_distance in round_trip_distances.items():
            feasible_pairs = [
                (pair, distance) for pair, distance in pairs_distance if distance <= max_distance.get(technician, float('inf'))
            ]
            tech_feasible_pairs[day] = feasible_pairs
        feasible_combinations[technician] = tech_feasible_pairs
    return feasible_combinations

# Calculate the round-trip distances
request_pairs_across_days = find_request_pairs_across_days(assignment_schedule, request_coordinates)
technician_round_trip_distances = calculate_round_trip_distances(home_coordinates, request_pairs_across_days, request_coordinates)

# Find the two pairs with the lowest distances for each technician on each day
lowest_distance_pairs = find_lowest_distance_pairs(technician_round_trip_distances)

# Determine if the request combinations can be handled within the max travel distance
feasible_combinations = can_handle_request_combinations(technician_round_trip_distances, max_distance)

# Print the feasible combinations for each technician
for technician, feasible_pairs in feasible_combinations.items():
    print(f"Technician {technician}:")
    for day, pairs_distance in feasible_pairs.items():
        print(f"  Assignment Day {day}:")
        for pair, distance in pairs_distance:
            request_id1, request_id2 = pair
            print(f"    Request Pair ({request_id1}, {request_id2}): Round-Trip Distance {distance}")

feasible_combinations



Technician 1:
  Assignment Day 1:
  Assignment Day 2:
  Assignment Day 3:
    Request Pair (1, 3): Round-Trip Distance 212
  Assignment Day 4:
    Request Pair (1, 3): Round-Trip Distance 212
    Request Pair (1, 4): Round-Trip Distance 212
    Request Pair (1, 5): Round-Trip Distance 408
    Request Pair (1, 6): Round-Trip Distance 408
    Request Pair (1, 8): Round-Trip Distance 408
    Request Pair (3, 4): Round-Trip Distance 212
    Request Pair (3, 5): Round-Trip Distance 461
    Request Pair (3, 6): Round-Trip Distance 461
    Request Pair (3, 8): Round-Trip Distance 461
    Request Pair (4, 5): Round-Trip Distance 461
    Request Pair (4, 6): Round-Trip Distance 461
    Request Pair (4, 8): Round-Trip Distance 461
    Request Pair (5, 6): Round-Trip Distance 408
    Request Pair (5, 8): Round-Trip Distance 408
    Request Pair (6, 8): Round-Trip Distance 408
  Assignment Day 5:
    Request Pair (1, 3): Round-Trip Distance 212
    Request Pair (1, 4): Round-Trip Distance 212
    

{1: {1: [],
  2: [],
  3: [((1, 3), 212)],
  4: [((1, 3), 212),
   ((1, 4), 212),
   ((1, 5), 408),
   ((1, 6), 408),
   ((1, 8), 408),
   ((3, 4), 212),
   ((3, 5), 461),
   ((3, 6), 461),
   ((3, 8), 461),
   ((4, 5), 461),
   ((4, 6), 461),
   ((4, 8), 461),
   ((5, 6), 408),
   ((5, 8), 408),
   ((6, 8), 408)],
  5: [((1, 3), 212),
   ((1, 4), 212),
   ((1, 5), 408),
   ((1, 6), 408),
   ((1, 8), 408),
   ((1, 9), 576),
   ((1, 10), 90),
   ((1, 11), 212),
   ((1, 12), 576),
   ((3, 4), 212),
   ((3, 5), 461),
   ((3, 6), 461),
   ((3, 8), 461),
   ((3, 9), 623),
   ((3, 10), 223),
   ((3, 11), 212),
   ((3, 12), 623),
   ((4, 5), 461),
   ((4, 6), 461),
   ((4, 8), 461),
   ((4, 9), 623),
   ((4, 10), 223),
   ((4, 11), 212),
   ((4, 12), 623),
   ((5, 6), 408),
   ((5, 8), 408),
   ((5, 9), 577),
   ((5, 10), 448),
   ((5, 11), 461),
   ((5, 12), 577),
   ((6, 8), 408),
   ((6, 9), 577),
   ((6, 10), 448),
   ((6, 11), 461),
   ((6, 12), 577),
   ((8, 9), 577),
   ((8, 10), 448),

In [1072]:
def filter_combinations_by_skill_set(feasible_combinations, doable_requests_copy):
    """
    Filter the feasible request combinations to include only those where both requests are in the technician's skill set.

    Args:
    - feasible_combinations: Dictionary where keys are technician IDs and values are dictionaries with keys as assignment days
      and values are lists of tuples ((request ID 1, request ID 2), round-trip distance).
    - doable_requests_copy: Dictionary where keys are technician IDs and values are lists of request IDs that the technician can handle.

    Returns:
    - skill_filtered_combinations: Dictionary where keys are technician IDs and values are dictionaries with keys as assignment days
      and values are lists of tuples ((request ID 1, request ID 2), round-trip distance) that are within the technician's skill set.
    """
    skill_filtered_combinations = {}
    for technician, day_pairs in feasible_combinations.items():
        tech_skill_filtered = {}
        for day, pairs_distance in day_pairs.items():
            filtered_pairs = [
                (pair, distance) for pair, distance in pairs_distance
                if pair[0] in doable_requests_copy.get(technician, []) and pair[1] in doable_requests_copy.get(technician, [])
            ]
            tech_skill_filtered[day] = filtered_pairs
        skill_filtered_combinations[technician] = tech_skill_filtered
    return skill_filtered_combinations

# Example usage:

# Define feasible combinations based on max distance
feasible_combinations = can_handle_request_combinations(technician_round_trip_distances, max_distance)


# Filter the feasible combinations by the skill set of each technician
skill_filtered_combinations = filter_combinations_by_skill_set(feasible_combinations, doable_requests_copy)

# Print the skill filtered combinations for each technician
for technician, filtered_pairs in skill_filtered_combinations.items():
    print(f"Technician {technician}:")
    for day, pairs_distance in filtered_pairs.items():
        print(f"  Assignment Day {day}:")
        for pair, distance in pairs_distance:
            request_id1, request_id2 = pair
            print(f"    Request Pair ({request_id1}, {request_id2}): Round-Trip Distance {distance}")

skill_filtered_combinations


Technician 1:
  Assignment Day 1:
  Assignment Day 2:
  Assignment Day 3:
  Assignment Day 4:
    Request Pair (5, 8): Round-Trip Distance 408
  Assignment Day 5:
    Request Pair (5, 8): Round-Trip Distance 408
    Request Pair (5, 11): Round-Trip Distance 461
    Request Pair (8, 11): Round-Trip Distance 461
  Assignment Day 6:
    Request Pair (5, 8): Round-Trip Distance 408
    Request Pair (5, 11): Round-Trip Distance 461
    Request Pair (5, 13): Round-Trip Distance 408
    Request Pair (8, 11): Round-Trip Distance 461
    Request Pair (8, 13): Round-Trip Distance 408
    Request Pair (11, 13): Round-Trip Distance 461
  Assignment Day 7:
    Request Pair (5, 8): Round-Trip Distance 408
    Request Pair (5, 11): Round-Trip Distance 461
    Request Pair (5, 13): Round-Trip Distance 408
    Request Pair (5, 14): Round-Trip Distance 408
    Request Pair (8, 11): Round-Trip Distance 461
    Request Pair (8, 13): Round-Trip Distance 408
    Request Pair (8, 14): Round-Trip Distance 408

{1: {1: [],
  2: [],
  3: [],
  4: [((5, 8), 408)],
  5: [((5, 8), 408), ((5, 11), 461), ((8, 11), 461)],
  6: [((5, 8), 408),
   ((5, 11), 461),
   ((5, 13), 408),
   ((8, 11), 461),
   ((8, 13), 408),
   ((11, 13), 461)],
  7: [((5, 8), 408),
   ((5, 11), 461),
   ((5, 13), 408),
   ((5, 14), 408),
   ((8, 11), 461),
   ((8, 13), 408),
   ((8, 14), 408),
   ((11, 13), 461),
   ((11, 14), 461),
   ((13, 14), 408)],
  8: [((5, 8), 408),
   ((5, 11), 461),
   ((5, 13), 408),
   ((5, 14), 408),
   ((5, 17), 448),
   ((5, 19), 577),
   ((8, 11), 461),
   ((8, 13), 408),
   ((8, 14), 408),
   ((8, 17), 448),
   ((8, 19), 577),
   ((11, 13), 461),
   ((11, 14), 461),
   ((11, 17), 223),
   ((11, 19), 623),
   ((13, 14), 408),
   ((13, 17), 448),
   ((13, 19), 577),
   ((14, 17), 448),
   ((14, 19), 577),
   ((17, 19), 615)],
  9: [((5, 8), 408),
   ((5, 11), 461),
   ((5, 13), 408),
   ((5, 14), 408),
   ((5, 17), 448),
   ((5, 19), 577),
   ((5, 21), 408),
   ((8, 11), 461),
   ((8, 13), 4

In [1073]:
def filter_combinations_by_max_installations(skill_filtered_combinations, machine_count, max_installations):
    """
    Filter the skill filtered combinations to include only those where the total machine count
    for the request pair does not exceed the technician's maximum installation capacity.

    Args:
    - skill_filtered_combinations: Dictionary where keys are technician IDs and values are dictionaries with keys as assignment days
      and values are lists of tuples ((request ID 1, request ID 2), round-trip distance) that are within the technician's skill set.
    - machine_count: Dictionary where keys are request IDs and values are the number of machines for each request.
    - max_installations: Dictionary where keys are technician IDs and values are the maximum number of machines the technician can install.

    Returns:
    - final_filtered_combinations: Dictionary where keys are technician IDs and values are dictionaries with keys as assignment days
      and values are lists of tuples ((request ID 1, request ID 2), round-trip distance) that are within the technician's skill set
      and within the maximum installation capacity.
    """
    final_filtered_combinations = {}
    for technician, day_pairs in skill_filtered_combinations.items():
        tech_filtered = {}
        for day, pairs_distance in day_pairs.items():
            filtered_pairs = [
                (pair, distance) for pair, distance in pairs_distance
                if machine_count[pair[0]] + machine_count[pair[1]] <= max_installations[technician]
            ]
            tech_filtered[day] = filtered_pairs
        final_filtered_combinations[technician] = tech_filtered
    return final_filtered_combinations

# Example usage:

# Define the skill filtered combinations
skill_filtered_combinations = filter_combinations_by_skill_set(feasible_combinations, doable_requests_copy)


# Filter the skill filtered combinations by the maximum installations for each technician
final_filtered_combinations = filter_combinations_by_max_installations(skill_filtered_combinations, machine_count, max_installations)

# Print the final filtered combinations for each technician
for technician, filtered_pairs in final_filtered_combinations.items():
    print(f"Technician {technician}:")
    for day, pairs_distance in filtered_pairs.items():
        print(f"  Assignment Day {day}:")
        for pair, distance in pairs_distance:
            request_id1, request_id2 = pair
            print(f"    Request Pair ({request_id1}, {request_id2}): Round-Trip Distance {distance}")

final_filtered_combinations


Technician 1:
  Assignment Day 1:
  Assignment Day 2:
  Assignment Day 3:
  Assignment Day 4:
    Request Pair (5, 8): Round-Trip Distance 408
  Assignment Day 5:
    Request Pair (5, 8): Round-Trip Distance 408
    Request Pair (5, 11): Round-Trip Distance 461
    Request Pair (8, 11): Round-Trip Distance 461
  Assignment Day 6:
    Request Pair (5, 8): Round-Trip Distance 408
    Request Pair (5, 11): Round-Trip Distance 461
    Request Pair (5, 13): Round-Trip Distance 408
    Request Pair (8, 11): Round-Trip Distance 461
    Request Pair (8, 13): Round-Trip Distance 408
    Request Pair (11, 13): Round-Trip Distance 461
  Assignment Day 7:
    Request Pair (5, 8): Round-Trip Distance 408
    Request Pair (5, 11): Round-Trip Distance 461
    Request Pair (5, 13): Round-Trip Distance 408
    Request Pair (5, 14): Round-Trip Distance 408
    Request Pair (8, 11): Round-Trip Distance 461
    Request Pair (8, 13): Round-Trip Distance 408
    Request Pair (8, 14): Round-Trip Distance 408

{1: {1: [],
  2: [],
  3: [],
  4: [((5, 8), 408)],
  5: [((5, 8), 408), ((5, 11), 461), ((8, 11), 461)],
  6: [((5, 8), 408),
   ((5, 11), 461),
   ((5, 13), 408),
   ((8, 11), 461),
   ((8, 13), 408),
   ((11, 13), 461)],
  7: [((5, 8), 408),
   ((5, 11), 461),
   ((5, 13), 408),
   ((5, 14), 408),
   ((8, 11), 461),
   ((8, 13), 408),
   ((8, 14), 408),
   ((11, 13), 461),
   ((11, 14), 461),
   ((13, 14), 408)],
  8: [((5, 8), 408),
   ((5, 11), 461),
   ((5, 13), 408),
   ((5, 14), 408),
   ((5, 17), 448),
   ((5, 19), 577),
   ((8, 11), 461),
   ((8, 13), 408),
   ((8, 14), 408),
   ((8, 17), 448),
   ((8, 19), 577),
   ((11, 13), 461),
   ((11, 14), 461),
   ((11, 17), 223),
   ((11, 19), 623),
   ((13, 14), 408),
   ((13, 17), 448),
   ((13, 19), 577),
   ((14, 17), 448),
   ((14, 19), 577),
   ((17, 19), 615)],
  9: [((5, 8), 408),
   ((5, 11), 461),
   ((5, 13), 408),
   ((5, 14), 408),
   ((5, 17), 448),
   ((5, 19), 577),
   ((5, 21), 408),
   ((8, 11), 461),
   ((8, 13), 4

In [1074]:
def can_handle_pairs_for_technicians(double_check_requests, request_pairs_across_days):
    """
    Check if each technician can handle both requests in all request pairs.

    Args:
    - double_check_requests: Dictionary where keys are technician IDs and values are lists of request IDs the technician can handle.
    - request_pairs_across_days: Dictionary where keys are days and values are lists of tuples ((request ID 1, request ID 2), distance).

    Returns:
    - technician_capabilities: Dictionary where keys are technician IDs and values are dictionaries.
      Each dictionary has keys as assignment days and values as lists of pairs that the technician can handle.
    """
    technician_capabilities = {}

    for technician_id, tech_requests in double_check_requests.items():
        technician_capabilities[technician_id] = {}
        for day, pairs_distance in request_pairs_across_days.items():
            request_pairs_for_technician = []
            for pair, distance in pairs_distance:
                if all(request in tech_requests for request in pair):
                    request_pairs_for_technician.append(pair)
            technician_capabilities[technician_id][day] = request_pairs_for_technician

    return technician_capabilities

# Example usage:
technician_capabilities = can_handle_pairs_for_technicians(double_check_requests, request_pairs_across_days)

# Print which technicians can handle which request pairs on which days
for technician, capabilities in technician_capabilities.items():
    print(f"Technician {technician}:")
    for day, pairs in capabilities.items():
        print(f"  Assignment Day {day}:")
        for pair in pairs:
            print(f"    Request Pair {pair}")

technician_capabilities


Technician 1:
  Assignment Day 1:
  Assignment Day 2:
  Assignment Day 3:
  Assignment Day 4:
    Request Pair (5, 8)
  Assignment Day 5:
    Request Pair (5, 8)
    Request Pair (5, 11)
    Request Pair (8, 11)
  Assignment Day 6:
    Request Pair (5, 8)
    Request Pair (5, 11)
    Request Pair (5, 13)
    Request Pair (8, 11)
    Request Pair (8, 13)
    Request Pair (11, 13)
  Assignment Day 7:
    Request Pair (5, 8)
    Request Pair (5, 11)
    Request Pair (5, 13)
    Request Pair (5, 14)
    Request Pair (8, 11)
    Request Pair (8, 13)
    Request Pair (8, 14)
    Request Pair (11, 13)
    Request Pair (11, 14)
    Request Pair (13, 14)
  Assignment Day 8:
    Request Pair (5, 8)
    Request Pair (5, 11)
    Request Pair (5, 13)
    Request Pair (5, 14)
    Request Pair (5, 17)
    Request Pair (5, 19)
    Request Pair (8, 11)
    Request Pair (8, 13)
    Request Pair (8, 14)
    Request Pair (8, 17)
    Request Pair (8, 19)
    Request Pair (11, 13)
    Request Pair (11, 14)


{1: {1: [],
  2: [],
  3: [],
  4: [(5, 8)],
  5: [(5, 8), (5, 11), (8, 11)],
  6: [(5, 8), (5, 11), (5, 13), (8, 11), (8, 13), (11, 13)],
  7: [(5, 8),
   (5, 11),
   (5, 13),
   (5, 14),
   (8, 11),
   (8, 13),
   (8, 14),
   (11, 13),
   (11, 14),
   (13, 14)],
  8: [(5, 8),
   (5, 11),
   (5, 13),
   (5, 14),
   (5, 17),
   (5, 19),
   (8, 11),
   (8, 13),
   (8, 14),
   (8, 17),
   (8, 19),
   (11, 13),
   (11, 14),
   (11, 17),
   (11, 19),
   (13, 14),
   (13, 17),
   (13, 19),
   (14, 17),
   (14, 19),
   (17, 19)],
  9: [(5, 8),
   (5, 11),
   (5, 13),
   (5, 14),
   (5, 17),
   (5, 19),
   (5, 21),
   (8, 11),
   (8, 13),
   (8, 14),
   (8, 17),
   (8, 19),
   (8, 21),
   (11, 13),
   (11, 14),
   (11, 17),
   (11, 19),
   (11, 21),
   (13, 14),
   (13, 17),
   (13, 19),
   (13, 21),
   (14, 17),
   (14, 19),
   (14, 21),
   (17, 19),
   (17, 21),
   (19, 21)],
  10: [(5, 8),
   (5, 11),
   (5, 13),
   (5, 14),
   (5, 17),
   (5, 19),
   (5, 21),
   (8, 11),
   (8, 13),
   (8

In [1075]:
print(double_check_requests)

{1: [5, 8, 11, 13, 14, 17, 19, 21, 28, 29], 2: [1, 21], 4: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 5: [1, 3, 4, 6, 7, 9, 10, 12, 15, 16, 18, 20, 23, 24, 25, 26, 27, 30], 6: [5, 8, 13, 14, 19, 28, 29], 7: [1, 2, 5, 6, 7, 8, 9, 12, 13, 14, 15, 19, 21, 22, 24, 25, 26, 28, 29], 8: [24, 26, 6, 7], 9: [7], 10: [6, 7, 9, 12, 15, 24, 25, 26]}


In [1076]:
print(request_pairs_across_days)

{1: [], 2: [], 3: [((1, 2), 564), ((1, 3), 106), ((2, 3), 571)], 4: [((1, 2), 564), ((1, 3), 106), ((1, 4), 106), ((1, 5), 204), ((1, 6), 204), ((1, 7), 564), ((1, 8), 204), ((2, 3), 571), ((2, 4), 571), ((2, 5), 435), ((2, 6), 435), ((2, 7), 0), ((2, 8), 435), ((3, 4), 0), ((3, 5), 151), ((3, 6), 151), ((3, 7), 571), ((3, 8), 151), ((4, 5), 151), ((4, 6), 151), ((4, 7), 571), ((4, 8), 151), ((5, 6), 0), ((5, 7), 435), ((5, 8), 0), ((6, 7), 435), ((6, 8), 0), ((7, 8), 435)], 5: [((1, 2), 564), ((1, 3), 106), ((1, 4), 106), ((1, 5), 204), ((1, 6), 204), ((1, 7), 564), ((1, 8), 204), ((1, 9), 288), ((1, 10), 45), ((1, 11), 106), ((1, 12), 288), ((2, 3), 571), ((2, 4), 571), ((2, 5), 435), ((2, 6), 435), ((2, 7), 0), ((2, 8), 435), ((2, 9), 396), ((2, 10), 588), ((2, 11), 571), ((2, 12), 396), ((3, 4), 0), ((3, 5), 151), ((3, 6), 151), ((3, 7), 571), ((3, 8), 151), ((3, 9), 229), ((3, 10), 72), ((3, 11), 0), ((3, 12), 229), ((4, 5), 151), ((4, 6), 151), ((4, 7), 571), ((4, 8), 151), ((4, 

In [1077]:
# method om te kijken of pairs in find_requests_with_same_day_and_loc are also pairs in doable_requests_double_check
def check_tech_req_same_day(double_check_requests, request_coordinates, assignment_schedule, location_ids):
    """
    Check if technicians can do requests with the same location and assignment day.

    Args:
    - double_check_requests: Dictionary where keys are technician IDs and values are lists of doable request IDs.
    - request_coordinates: Dictionary where keys are request IDs and values are tuples representing location coordinates.
    - assignment_schedule: Dictionary where keys are request IDs and values are assignment days.
    - location_ids: List of location IDs extracted from the requests.

    Returns:
    - tech_req_same_day: Dictionary where keys are tuples (technician ID, assignment day) and values are lists of request IDs.
    """
    tech_req_same_day = {}

    # Iterate through technician IDs
    for tech_id, requests in double_check_requests.items():
        # Initialize a dictionary to store requests with the same location and assignment day for the technician
        req_same_day = {}

        # Iterate over the technician's double-check requests
        for req_id in requests:
            location_id = location_ids[req_id - 1]  # Adjusting index since req_id starts from 1
            day = assignment_schedule[req_id]

            # Create a key for the location and assignment day
            key = (location_id, day)

            # Add the request to the dictionary under the corresponding key
            req_same_day.setdefault(key, []).append(req_id)

        # Filter out requests with the same location and assignment day
        req_same_day = {key: req_ids for key, req_ids in req_same_day.items() if len(req_ids) > 1}

        # Add the filtered dictionary to the tech_req_same_day dictionary
        if req_same_day:
            tech_req_same_day[tech_id] = req_same_day

    return tech_req_same_day

# Example usage:
tech_req_same_day = check_tech_req_same_day(double_check_requests, request_coordinates, assignment_schedule, location_ids)

# Print technicians who can do requests with the same location and assignment day
print("Technicians and Requests with the Same Location ID and Assignment Day:")
for tech_id, req_same_day in tech_req_same_day.items():
    for (location_id, assignment_day), req_ids in req_same_day.items():
        print(f"Technician ID {tech_id}, Location ID {location_id}, Assignment Day {assignment_day}: Requests {req_ids}")


Technicians and Requests with the Same Location ID and Assignment Day:
Technician ID 1, Location ID 7, Assignment Day 4: Requests [5, 8]
Technician ID 4, Location ID 7, Assignment Day 4: Requests [5, 6, 8]
Technician ID 4, Location ID 6, Assignment Day 5: Requests [9, 12]
Technician ID 4, Location ID 5, Assignment Day 8: Requests [16, 17, 18]
Technician ID 5, Location ID 6, Assignment Day 5: Requests [9, 12]
Technician ID 5, Location ID 5, Assignment Day 8: Requests [16, 18]
Technician ID 6, Location ID 7, Assignment Day 4: Requests [5, 8]
Technician ID 7, Location ID 7, Assignment Day 4: Requests [5, 6, 8]
Technician ID 7, Location ID 6, Assignment Day 5: Requests [9, 12]
Technician ID 10, Location ID 6, Assignment Day 5: Requests [9, 12]


In [1078]:
def possible_tech_combis_on_day(tech_req_same_day):
    """
    Create pairs of requests for technicians who can handle requests with the same location ID and assignment day.

    Args:
    - tech_req_same_day: Dictionary where keys are technician IDs and values are dictionaries of requests with the same location and assignment day.

    Returns:
    - tech_combis: List of tuples where each tuple contains a pair of requests for a technician.
    """
    tech_combis = []

    # Iterate through technicians
    for tech_id, req_same_day in tech_req_same_day.items():
        # Iterate through requests with the same location and assignment day
        for req_ids in req_same_day.values():
            # Create pairs of consecutive requests
            for i in range(len(req_ids) - 1):
                tech_combis.append((tech_id, req_ids[i], req_ids[i + 1]))

    return tech_combis

# Example usage:
tech_combis = possible_tech_combis_on_day(tech_req_same_day)

# Print possible technician pairs with consecutive requests
print("Possible Technician Pairs with Consecutive Requests:")
for tech_id, req1, req2 in tech_combis:
    print(f"Technician ID {tech_id}: Requests {req1} and {req2}")
    
tech_combis


Possible Technician Pairs with Consecutive Requests:
Technician ID 1: Requests 5 and 8
Technician ID 4: Requests 5 and 6
Technician ID 4: Requests 6 and 8
Technician ID 4: Requests 9 and 12
Technician ID 4: Requests 16 and 17
Technician ID 4: Requests 17 and 18
Technician ID 5: Requests 9 and 12
Technician ID 5: Requests 16 and 18
Technician ID 6: Requests 5 and 8
Technician ID 7: Requests 5 and 6
Technician ID 7: Requests 6 and 8
Technician ID 7: Requests 9 and 12
Technician ID 10: Requests 9 and 12


[(1, 5, 8),
 (4, 5, 6),
 (4, 6, 8),
 (4, 9, 12),
 (4, 16, 17),
 (4, 17, 18),
 (5, 9, 12),
 (5, 16, 18),
 (6, 5, 8),
 (7, 5, 6),
 (7, 6, 8),
 (7, 9, 12),
 (10, 9, 12)]

In [1079]:
def weighted_tech_combis(tech_combis, machine_count):
    """
    Calculate the weight of each tech combi based on the machine count of the requests it contains.

    Args:
    - tech_combis: List of tuples where each tuple contains a tech ID and two request IDs.
    - machine_count: Dictionary where keys are request IDs and values are the machine count.

    Returns:
    - weighted_combis: Dictionary where keys are tech combis (tuples) and values are their weights.
    """
    weighted_combis = {}
    
    # Iterate through each tech combi
    for tech_id, req1, req2 in tech_combis:
        # Calculate the weight of the tech combi by summing the machine counts of the requests it contains
        weight = machine_count.get(req1, 0) + machine_count.get(req2, 0)
        weighted_combis[(tech_id, req1, req2)] = weight
    
    return weighted_combis

# Example usage:
weighted_combis = weighted_tech_combis(tech_combis, machine_count)

# Print weighted tech combis
print("Weighted Tech Combis:")
for combi, weight in weighted_combis.items():
    print(f"Tech Combi {combi}: Weight {weight}")

weighted_combis

Weighted Tech Combis:
Tech Combi (1, 5, 8): Weight 3
Tech Combi (4, 5, 6): Weight 3
Tech Combi (4, 6, 8): Weight 4
Tech Combi (4, 9, 12): Weight 4
Tech Combi (4, 16, 17): Weight 3
Tech Combi (4, 17, 18): Weight 2
Tech Combi (5, 9, 12): Weight 4
Tech Combi (5, 16, 18): Weight 3
Tech Combi (6, 5, 8): Weight 3
Tech Combi (7, 5, 6): Weight 3
Tech Combi (7, 6, 8): Weight 4
Tech Combi (7, 9, 12): Weight 4
Tech Combi (10, 9, 12): Weight 4


{(1, 5, 8): 3,
 (4, 5, 6): 3,
 (4, 6, 8): 4,
 (4, 9, 12): 4,
 (4, 16, 17): 3,
 (4, 17, 18): 2,
 (5, 9, 12): 4,
 (5, 16, 18): 3,
 (6, 5, 8): 3,
 (7, 5, 6): 3,
 (7, 6, 8): 4,
 (7, 9, 12): 4,
 (10, 9, 12): 4}

In [1080]:
def feasible_tech_combi(weighted_combis, max_installations):
    """
    Check if each tech combi is feasible for a technician based on their maximum installations per day.

    Args:
    - weighted_combis: Dictionary with tech combis as keys and their weights as values.
    - max_installations: Dictionary where keys are technician IDs and values are their maximum installations.

    Returns:
    - feasible_combis: Dictionary with tech combis as keys and boolean values indicating feasibility.
    """
    feasible_combis = {}
    for tech_combi, combi_weight in weighted_combis.items():
        tech_id = tech_combi[0]
        max_installation = max_installations.get(tech_id, 0)
        feasible_combis[tech_combi] = combi_weight <= max_installation
    return feasible_combis


feasible_combis = feasible_tech_combi(weighted_combis, max_installations)

# Print the feasibility of each tech combi
for tech_combi, is_feasible in feasible_combis.items():
    print(f"Is tech combi {tech_combi} feasible? {is_feasible}")


Is tech combi (1, 5, 8) feasible? True
Is tech combi (4, 5, 6) feasible? True
Is tech combi (4, 6, 8) feasible? True
Is tech combi (4, 9, 12) feasible? True
Is tech combi (4, 16, 17) feasible? True
Is tech combi (4, 17, 18) feasible? True
Is tech combi (5, 9, 12) feasible? True
Is tech combi (5, 16, 18) feasible? True
Is tech combi (6, 5, 8) feasible? True
Is tech combi (7, 5, 6) feasible? True
Is tech combi (7, 6, 8) feasible? True
Is tech combi (7, 9, 12) feasible? True
Is tech combi (10, 9, 12) feasible? True


In [1081]:
final_filtered_combinations_dict = {}

for tech_id, days_combinations in final_filtered_combinations.items():
    for day, pairs_distances in days_combinations.items():
        for pair_distance in pairs_distances:
            if pair_distance:
                req1, req2 = pair_distance[0]
                final_filtered_combinations_dict[(tech_id, req1, req2)] = True
print(final_filtered_combinations_dict)

{(1, 5, 8): True, (1, 5, 11): True, (1, 8, 11): True, (1, 5, 13): True, (1, 8, 13): True, (1, 11, 13): True, (1, 5, 14): True, (1, 8, 14): True, (1, 11, 14): True, (1, 13, 14): True, (1, 5, 17): True, (1, 5, 19): True, (1, 8, 17): True, (1, 8, 19): True, (1, 11, 17): True, (1, 11, 19): True, (1, 13, 17): True, (1, 13, 19): True, (1, 14, 17): True, (1, 14, 19): True, (1, 17, 19): True, (1, 5, 21): True, (1, 8, 21): True, (1, 11, 21): True, (1, 13, 21): True, (1, 14, 21): True, (1, 17, 21): True, (1, 19, 21): True, (1, 5, 28): True, (1, 5, 29): True, (1, 8, 28): True, (1, 8, 29): True, (1, 11, 28): True, (1, 11, 29): True, (1, 13, 28): True, (1, 13, 29): True, (1, 14, 28): True, (1, 14, 29): True, (1, 17, 28): True, (1, 17, 29): True, (1, 19, 28): True, (1, 19, 29): True, (1, 21, 28): True, (1, 21, 29): True, (1, 28, 29): True, (2, 1, 21): True, (4, 1, 3): True, (4, 1, 4): True, (4, 1, 5): True, (4, 1, 6): True, (4, 1, 8): True, (4, 2, 7): True, (4, 3, 4): True, (4, 3, 5): True, (4, 3, 6

In [1082]:
def add_third_request_to_pairs(final_filtered_combinations_dict, location_ids, machine_count, max_installations):
    triple_combinations_dict = {}
    
    for (tech_id, req1, req2) in final_filtered_combinations_dict.keys():
        location1 = location_ids[req1 - 1]
        location2 = location_ids[req2 - 1]
        
        for req3, loc3 in enumerate(location_ids, start=1):
            if req3 in (req1, req2):
                continue
            
            if machine_count[req1] + machine_count[req2] + machine_count[req3] > max_installations[tech_id]:
                continue
            
            if loc3 == location1:
                triple_combinations_dict[(tech_id, req1, req3, req2)] = True
            elif loc3 == location2:
                triple_combinations_dict[(tech_id, req1, req2, req3)] = True

    return triple_combinations_dict

# Example usage
triple_combinations_dict = add_third_request_to_pairs(final_filtered_combinations_dict, location_ids, machine_count, max_installations)

# Print new combinations
print("New Combinations with Third Request Added:")
for combi in triple_combinations_dict.keys():
    print(f"Technician {combi[0]}: Requests {combi[1]}, {combi[2]}, and {combi[3]}")


New Combinations with Third Request Added:
Technician 1: Requests 5, 6, and 8
Technician 1: Requests 5, 13, and 8
Technician 1: Requests 5, 14, and 8
Technician 1: Requests 5, 26, and 8
Technician 1: Requests 5, 28, and 8
Technician 1: Requests 5, 11, and 3
Technician 1: Requests 5, 11, and 4
Technician 1: Requests 5, 6, and 11
Technician 1: Requests 5, 8, and 11
Technician 1: Requests 5, 13, and 11
Technician 1: Requests 5, 14, and 11
Technician 1: Requests 5, 11, and 20
Technician 1: Requests 5, 26, and 11
Technician 1: Requests 5, 11, and 27
Technician 1: Requests 5, 28, and 11
Technician 1: Requests 5, 11, and 30
Technician 1: Requests 8, 11, and 3
Technician 1: Requests 8, 11, and 4
Technician 1: Requests 8, 5, and 11
Technician 1: Requests 8, 6, and 11
Technician 1: Requests 8, 13, and 11
Technician 1: Requests 8, 14, and 11
Technician 1: Requests 8, 11, and 20
Technician 1: Requests 8, 26, and 11
Technician 1: Requests 8, 11, and 27
Technician 1: Requests 8, 28, and 11
Technicia

In [1083]:
# read again, omdat bij de tech assignments de request array weg gaat
"""
Enter filepath of instance and desired name for the solution file
"""
directory = "/Users/juliasmit/Documents/Business Analytics/jaar 3/Combinatorial Optimization/project"
filename = filename
output_file_name = output_file_name


"""
 Read the instance and assign variables to the different values
"""

data = pd.read_csv(filename, sep='\t+', header=None, engine='python')
data = data[0].str.split(expand=True)
title = data.iloc[1,2] + ' ' + data.iloc[1,3]
maxdays = data.iloc[2,2]
maxdays = int(maxdays)
constraints = data[[0, 2]].iloc[2:5]
constraints = constraints.set_index(0)[2].to_dict()
constraints = dict((k, int(v)) for k, v in constraints.items())
costs = data[[0, 2]].iloc[5:11]
costs = costs.set_index(0)[2].to_dict()
costs = dict((k, int(v)) for k, v in costs.items())
numberMachines = int(data.iloc[11][2])
index = (11 + numberMachines + 1)  # extra step that ensures the right values are taken
machines = np.array(data[[0, 1, 2]][12:index].astype(int))
numberLocations = int(data.iloc[index][2])
locations = np.array(data[[0, 1, 2]][index + 1:(index + numberLocations + 1)].astype(int))
index += numberLocations + 1  # extra step that ensures the right values are taken
depot = locations[0]
numberRequests = int(data.iloc[index][2])
requests = np.array(data[[0, 1, 2, 3, 4, 5]][index + 1:(index + numberRequests + 1)].astype(int))
index += numberRequests + 1  # extra step that ensures the right values are taken
numberTechnicians = int(data.iloc[index][2])
technicians = np.array(
        data[[i for i in range(0, numberMachines + 4)]][index + 1:(index + numberTechnicians + 1)].astype(int))
constraints

DAYS = constraints['DAYS']
TRUCK_CAPACITY = constraints['TRUCK_CAPACITY']
TRUCK_MAX_DISTANCE = constraints['TRUCK_MAX_DISTANCE']

TRUCK_DISTANCE_COST = costs['TRUCK_DISTANCE_COST']
TRUCK_DAY_COST = costs['TRUCK_DAY_COST']
TRUCK_COST = costs['TRUCK_COST']
TECHNICIAN_DISTANCE_COST = costs['TECHNICIAN_DISTANCE_COST']
TECHNICIAN_DAY_COST = costs['TECHNICIAN_DAY_COST']
TECHNICIAN_COST = costs['TECHNICIAN_COST']
IDLE_MACHINE_COSTS = {machine[0]: machine[2] for machine in machines}


In [1084]:
def time_window_for_requests(requests):
    time_windows = {}
    for request_info in requests:
        request_id = request_info[0]
        start_day = request_info[2]
        end_day = request_info[3]
        duration = end_day - start_day + 1  # Calculate duration in days
        time_windows[request_id] = duration
    return time_windows


time_windows = time_window_for_requests(requests)
print(time_windows)


{1: 3, 2: 4, 3: 2, 4: 2, 5: 2, 6: 4, 7: 3, 8: 3, 9: 3, 10: 4, 11: 2, 12: 4, 13: 2, 14: 3, 15: 2, 16: 4, 17: 3, 18: 4, 19: 4, 20: 3, 21: 2, 22: 3, 23: 4, 24: 4, 25: 4, 26: 3, 27: 3, 28: 4, 29: 3, 30: 3}


In [1085]:
def technician_rare_request_count(double_check_requests, rare_requests):
    # Initialize a dictionary to store the rare request count for each technician
    technician_rare_counts = {tech_id: 0 for tech_id in double_check_requests.keys()}
    
    # Iterate through rare requests and update the count for each technician
    for req_id, data in rare_requests.items():
        technicians = data['technicians']
        for tech_id in technicians:
            technician_rare_counts[tech_id] += 1
    
    return technician_rare_counts

In [1086]:
# Assuming technician_rare_requests is already defined somewhere before this point
# For example:
# technician_rare_requests = {tech_id: initial_value for tech_id in technician_ids}

# Ensure technician_rare_requests is defined and populated correctly
technician_rare_requests = {tech_id: 0 for tech_id in double_check_requests.keys()}

def update_available_technicians(current_day, available_technicians, technician_rest_days, consecutive_days, last_day_worked, technician_rest_days_details):
    for tech_id in list(available_technicians):
        rest_days_remaining = technician_rest_days.get(tech_id, 0)
        consecutive_days_worked = consecutive_days.get(tech_id, 0)
        last_day = last_day_worked.get(tech_id, None)
        rest_day_1, rest_day_2 = technician_rest_days_details.get(tech_id, (None, None))

        # Calculate rest_days_remaining based on current day and rest days
        if rest_day_1 is not None and current_day == rest_day_1:
            rest_days_remaining = 2
        elif rest_day_2 is not None and current_day == rest_day_2:
            rest_days_remaining = 1
        elif rest_day_2 is not None and current_day > rest_day_2:
            rest_days_remaining = 0

        technician_rest_days[tech_id] = rest_days_remaining

        print(f"Technician {tech_id} - Rest days remaining: {rest_days_remaining}, Consecutive days worked: {consecutive_days_worked}, Last day worked: {last_day}, Current day: {current_day}, Rest days: {rest_day_1}, {rest_day_2}")

        if (rest_day_1 is not None and rest_day_1 == current_day) or (rest_day_2 is not None and rest_day_2 == current_day):
            if tech_id in available_technicians:
                available_technicians.remove(tech_id)
            print(f"Technician {tech_id} is on a rest day {current_day}.")
            continue

        if rest_days_remaining > 0:
            if current_day in (rest_day_1, rest_day_2):
                technician_rest_days[tech_id] -= 1
                print(f"Technician {tech_id} is on rest day {rest_days_remaining}.")
                if tech_id in available_technicians:
                    available_technicians.remove(tech_id)
            else:
                print(f"Technician {tech_id} is available but has {rest_days_remaining} rest days remaining.")
        else:
            if consecutive_days_worked >= 4:
                technician_rest_days[tech_id] = 2
                consecutive_days[tech_id] = 0
                technician_rest_days_details[tech_id] = (current_day, current_day + 1)
                print(f"Technician {tech_id} has worked 4 consecutive days and now requires 2 rest days.")
                if tech_id in available_technicians:
                    available_technicians.remove(tech_id)
            else:
                print(f"Technician {tech_id} is available.")

def try_assign_request(request_id, day_to_assign, available_technicians, assigned_requests, assigned_request_ids, consecutive_days, last_day_worked, later_deliveries, final_filtered_combinations_dict, triple_combinations_dict, technician_rare_requests, technician_rest_days, assignment_schedule, double_check_requests):
    available_technicians = {tech_id for tech_id in available_technicians if technician_rest_days.get(tech_id, 0) == 0}
    print(f"\nTrying to assign Request {request_id} on Day {day_to_assign}")
    print(f"Available technicians after checking rest days: {available_technicians}")

    if not available_technicians:
        print("No available technicians.")
        return False

    min_rare_count = min(technician_rare_requests.get(tech_id, 0) for tech_id in available_technicians)
    min_consecutive_days = min(consecutive_days.get(tech_id, 0) for tech_id in available_technicians if technician_rare_requests.get(tech_id, 0) == min_rare_count)
    suitable_technicians = [tech_id for tech_id in available_technicians if technician_rare_requests.get(tech_id, 0) == min_rare_count and consecutive_days.get(tech_id, 0) == min_consecutive_days]

    if not suitable_technicians:
        print("No suitable technicians.")
        return False

    min_assigned = min(suitable_technicians, key=lambda tech_id: (technician_rare_requests.get(tech_id, 0), consecutive_days.get(tech_id, 0)))
    print(f"Selected Technician {min_assigned}")

    # Check the number of requests already assigned to the technician on the same day
    requests_assigned_today = [request for request in assigned_requests.get(min_assigned, []) if request[1] == day_to_assign]
    num_requests_assigned_today = len(requests_assigned_today)

    if num_requests_assigned_today >= 3:
        print(f"Technician {min_assigned} has already been assigned 3 requests on day {day_to_assign}.")
        return False

    if num_requests_assigned_today == 0:
        if request_id in double_check_requests[min_assigned]:
            print(f"Assigning single request {request_id} to technician {min_assigned} on day {day_to_assign}.")
            assigned_requests.setdefault(min_assigned, []).append((request_id, day_to_assign))
            assigned_request_ids.add(request_id)
        else:
            print(f"Single request {request_id} is not feasible for technician {min_assigned}.")
            return False

    elif num_requests_assigned_today == 1:
        existing_request = requests_assigned_today[0][0]
        if (min_assigned, min(existing_request, request_id), max(existing_request, request_id)) in final_filtered_combinations_dict:
            print(f"Assigning pair request {request_id} to technician {min_assigned} on day {day_to_assign}.")
            assigned_requests[min_assigned].append((request_id, day_to_assign))
            assigned_request_ids.add(request_id)
        else:
            print(f"Pair request {request_id} is not feasible with existing request {existing_request} for technician {min_assigned}.")
            return False

    elif num_requests_assigned_today == 2:
        existing_requests = [request[0] for request in requests_assigned_today]
        if (min_assigned, *sorted(existing_requests + [request_id])) in triple_combinations_dict:
            print(f"Assigning triple request {request_id} to technician {min_assigned} on day {day_to_assign}.")
            assigned_requests[min_assigned].append((request_id, day_to_assign))
            assigned_request_ids.add(request_id)
        else:
            print(f"Triple request {request_id} is not feasible with existing requests {existing_requests} for technician {min_assigned}.")
            return False

    # Update consecutive days and last day worked
    if last_day_worked.get(min_assigned) is not None:
        if day_to_assign == last_day_worked[min_assigned] + 1:
            consecutive_days[min_assigned] += 1
        elif day_to_assign != last_day_worked[min_assigned]:
            consecutive_days[min_assigned] = 1  # Reset if non-consecutive day
    last_day_worked[min_assigned] = day_to_assign

    if isinstance(assignment_schedule[request_id], list) and len(assignment_schedule[request_id]) > 3:
        if day_to_assign > assignment_schedule[request_id][3]:  # Assuming the deadline day is at index 3
            later_deliveries[request_id] = day_to_assign - assignment_schedule[request_id][3]

    return True

def assign_requests(assignment_schedule, double_check_requests, days, technician_rare_requests, final_filtered_combinations_dict, triple_combinations_dict):
    assigned_requests = {}
    assigned_request_ids = set()
    unassigned_requests = set()
    later_deliveries = {}
    consecutive_days = {tech_id: 0 for tech_id in double_check_requests.keys()}
    last_day_worked = {tech_id: None for tech_id in double_check_requests.keys()}
    technician_rest_days = {tech_id: 0 for tech_id in double_check_requests.keys()}
    technician_rest_days_details = {tech_id: (None, None) for tech_id in double_check_requests.keys()}

    for request_id, assignment_day in sorted(assignment_schedule.items(), key=lambda x: x[1]):
        print(f"\nProcessing Request {request_id} for Assignment Day {assignment_day}")
        if request_id in assigned_request_ids:
            print(f"Request {request_id} is already assigned.")
            continue

        assigned = False
        for day_delay in range(days + 1):
            day_to_assign = assignment_day + day_delay
            if day_to_assign > days:
                unassigned_requests.add(request_id)
                print(f"Request {request_id} could not be assigned within {days} days.")
                break

            available_technicians = {tech_id for tech_id, requests in double_check_requests.items() if request_id in requests}
            update_available_technicians(day_to_assign, available_technicians, technician_rest_days, consecutive_days, last_day_worked, technician_rest_days_details)

            if not available_technicians:
                print(f"No available technicians for Request {request_id} on Day {day_to_assign}")
                continue

            if try_assign_request(request_id, day_to_assign, available_technicians, assigned_requests, assigned_request_ids, consecutive_days, last_day_worked, later_deliveries, final_filtered_combinations_dict, triple_combinations_dict, technician_rare_requests, technician_rest_days, assignment_schedule, double_check_requests):
                assigned = True
                break

        if not assigned:
            unassigned_requests.add(request_id)
            print(f"Request {request_id} remains unassigned.")

    return assigned_requests, unassigned_requests, later_deliveries

# Ensure that technician_rare_requests is initialized before calling assign_requests
technician_rare_requests = {tech_id: 0 for tech_id in double_check_requests.keys()}

# Example usage
assigned_requests, unassigned_requests, later_deliveries = assign_requests(assignment_schedule, double_check_requests, DAYS, technician_rare_requests, final_filtered_combinations_dict, triple_combinations_dict)

# Print the assigned requests
for technician, requests in assigned_requests.items():
    print(f"\nTechnician {technician} assigned requests:")
    for request, day in requests:
        print(f"Request {request} assigned on day {day}")

# Print the later deliveries
print("\nLater deliveries:")
for request_id, delay in later_deliveries.items():
    print(f"Request {request_id} assigned {delay} day(s) later")

# Print the unassigned requests
print("\nUnassigned requests:")
for request_id in unassigned_requests:
    print(f"Request {request_id}")



Processing Request 1 for Assignment Day 3
Technician 2 - Rest days remaining: 0, Consecutive days worked: 0, Last day worked: None, Current day: 3, Rest days: None, None
Technician 2 is available.
Technician 4 - Rest days remaining: 0, Consecutive days worked: 0, Last day worked: None, Current day: 3, Rest days: None, None
Technician 4 is available.
Technician 5 - Rest days remaining: 0, Consecutive days worked: 0, Last day worked: None, Current day: 3, Rest days: None, None
Technician 5 is available.
Technician 7 - Rest days remaining: 0, Consecutive days worked: 0, Last day worked: None, Current day: 3, Rest days: None, None
Technician 7 is available.

Trying to assign Request 1 on Day 3
Available technicians after checking rest days: {2, 4, 5, 7}
Selected Technician 2
Assigning single request 1 to technician 2 on day 3.

Processing Request 2 for Assignment Day 3
Technician 4 - Rest days remaining: 0, Consecutive days worked: 0, Last day worked: None, Current day: 3, Rest days: None

In [1087]:
print(feasible_combis)
print(final_filtered_combinations)
print(final_filtered_combinations_dict)

{(1, 5, 8): True, (4, 5, 6): True, (4, 6, 8): True, (4, 9, 12): True, (4, 16, 17): True, (4, 17, 18): True, (5, 9, 12): True, (5, 16, 18): True, (6, 5, 8): True, (7, 5, 6): True, (7, 6, 8): True, (7, 9, 12): True, (10, 9, 12): True}
{1: {1: [], 2: [], 3: [], 4: [((5, 8), 408)], 5: [((5, 8), 408), ((5, 11), 461), ((8, 11), 461)], 6: [((5, 8), 408), ((5, 11), 461), ((5, 13), 408), ((8, 11), 461), ((8, 13), 408), ((11, 13), 461)], 7: [((5, 8), 408), ((5, 11), 461), ((5, 13), 408), ((5, 14), 408), ((8, 11), 461), ((8, 13), 408), ((8, 14), 408), ((11, 13), 461), ((11, 14), 461), ((13, 14), 408)], 8: [((5, 8), 408), ((5, 11), 461), ((5, 13), 408), ((5, 14), 408), ((5, 17), 448), ((5, 19), 577), ((8, 11), 461), ((8, 13), 408), ((8, 14), 408), ((8, 17), 448), ((8, 19), 577), ((11, 13), 461), ((11, 14), 461), ((11, 17), 223), ((11, 19), 623), ((13, 14), 408), ((13, 17), 448), ((13, 19), 577), ((14, 17), 448), ((14, 19), 577), ((17, 19), 615)], 9: [((5, 8), 408), ((5, 11), 461), ((5, 13), 408), 

In [1088]:
final_filtered_combinations_dict = {}

# Transform final_filtered_combinations into the desired format
for tech_id, days_combinations in final_filtered_combinations.items():
    for day, pairs_distances in days_combinations.items():
        for pair_distance in pairs_distances:
            if pair_distance:
                req1, req2 = pair_distance[0]
                final_filtered_combinations_dict[(tech_id, req1, req2)] = True


In [1089]:
#### inn the code hierboven niet alleen de tech_combis doen maar juist de final_filtered_combinatins

In [1090]:
def idle_request_days(later_deliveries):
    """
    Calculate the number of days each request is assigned later than its scheduled day.

    Args:
    - later_deliveries: Dictionary where keys are request IDs and values are the number of days delayed.

    Returns:
    - idle_days: Dictionary where keys are request IDs and values are the number of days each request is late.
    """
    idle_days = {}
    for request_id, delay in later_deliveries.items():
        idle_days[request_id] = delay

    return idle_days


# Example usage with the previous results
idle_days = idle_request_days(later_deliveries)

# Print idle request days
print("\nIdle request days:")
for request_id, days_late in idle_days.items():
    print(f"Request {request_id} is {days_late} day(s) too late")

idle_days



Idle request days:


{}

In [1091]:
# read again, omdat bij de tech assignments de request array weg gaat
"""
Enter filepath of instance and desired name for the solution file
"""
directory = "/Users/juliasmit/Documents/Business Analytics/jaar 3/Combinatorial Optimization/project"
filename = filename
output_file_name = output_file_name


"""
 Read the instance and assign variables to the different values
"""

data = pd.read_csv(filename, sep='\t+', header=None, engine='python')
data = data[0].str.split(expand=True)
title = data.iloc[1,2] + ' ' + data.iloc[1,3]
maxdays = data.iloc[2,2]
maxdays = int(maxdays)
constraints = data[[0, 2]].iloc[2:5]
constraints = constraints.set_index(0)[2].to_dict()
constraints = dict((k, int(v)) for k, v in constraints.items())
costs = data[[0, 2]].iloc[5:11]
costs = costs.set_index(0)[2].to_dict()
costs = dict((k, int(v)) for k, v in costs.items())
numberMachines = int(data.iloc[11][2])
index = (11 + numberMachines + 1)  # extra step that ensures the right values are taken
machines = np.array(data[[0, 1, 2]][12:index].astype(int))
numberLocations = int(data.iloc[index][2])
locations = np.array(data[[0, 1, 2]][index + 1:(index + numberLocations + 1)].astype(int))
index += numberLocations + 1  # extra step that ensures the right values are taken
depot = locations[0]
numberRequests = int(data.iloc[index][2])
requests = np.array(data[[0, 1, 2, 3, 4, 5]][index + 1:(index + numberRequests + 1)].astype(int))
index += numberRequests + 1  # extra step that ensures the right values are taken
numberTechnicians = int(data.iloc[index][2])
technicians = np.array(
        data[[i for i in range(0, numberMachines + 4)]][index + 1:(index + numberTechnicians + 1)].astype(int))
constraints

DAYS = constraints['DAYS']
TRUCK_CAPACITY = constraints['TRUCK_CAPACITY']
TRUCK_MAX_DISTANCE = constraints['TRUCK_MAX_DISTANCE']

TRUCK_DISTANCE_COST = costs['TRUCK_DISTANCE_COST']
TRUCK_DAY_COST = costs['TRUCK_DAY_COST']
TRUCK_COST = costs['TRUCK_COST']
TECHNICIAN_DISTANCE_COST = costs['TECHNICIAN_DISTANCE_COST']
TECHNICIAN_DAY_COST = costs['TECHNICIAN_DAY_COST']
TECHNICIAN_COST = costs['TECHNICIAN_COST']
IDLE_MACHINE_COSTS = {machine[0]: machine[2] for machine in machines}

print("Machines:")
print(machines)

print("\nLocations:")
print(locations)

print("\nRequests:")
print(requests)

print("\nTechnicians:")
print(technicians)


print(DAYS)
print(TRUCK_CAPACITY)
print(TRUCK_MAX_DISTANCE)

print(TRUCK_DISTANCE_COST)
print(TRUCK_DAY_COST)
print(TRUCK_COST)
print(TECHNICIAN_DISTANCE_COST)
print(TECHNICIAN_DAY_COST)
print(TECHNICIAN_COST)



Machines:
[[  1   6 892]
 [  2   7 358]
 [  3   7 844]]

Locations:
[[  1 356 237]
 [  2 369 577]
 [  3 274 532]
 [  4 533  31]
 [  5 298 569]
 [  6 541 426]
 [  7 464 460]
 [  8 234 340]
 [  9 206 343]]

Requests:
[[ 1  3  2  4  2  1]
 [ 2  4  2  5  3  2]
 [ 3  2  2  3  2  2]
 [ 4  2  3  4  2  1]
 [ 5  7  3  4  3  1]
 [ 6  7  3  6  1  2]
 [ 7  4  3  5  1  2]
 [ 8  7  3  5  3  2]
 [ 9  6  4  6  2  2]
 [10  5  4  7  2  1]
 [11  2  4  5  3  1]
 [12  6  4  7  2  2]
 [13  7  5  6  3  2]
 [14  7  6  8  3  1]
 [15  6  6  7  2  1]
 [16  5  7 10  1  2]
 [17  5  7  9  3  1]
 [18  5  7 10  1  1]
 [19  6  7 10  3  1]
 [20  2  8 10  2  2]
 [21  3  8  9  3  1]
 [22  4  8 10  3  1]
 [23  5  9 12  2  2]
 [24  6  9 12  1  2]
 [25  4  9 12  2  1]
 [26  7  9 11  1  2]
 [27  2 10 12  1  1]
 [28  7 10 13  3  1]
 [29  6 10 12  3  2]
 [30  2 11 13  1  2]]

Technicians:
[[  1   3 878   8   0   0   1]
 [  2   8 463   5   1   1   1]
 [  3   9 266   5   0   0   1]
 [  4   1 934   6   1   1   1]
 [  5   1 956   

In [1092]:
def get_idle_days_per_machine(idle_days, requests):
    """
    Calculate the total idle days for each machine type.

    Args:
    - idle_days: Dictionary where keys are request IDs and values are the number of days each request is late.
    - requests: List of requests where each request is a tuple.

    Returns:
    - machine_idle_days: Dictionary where keys are machine types and values are the total number of idle days.
    """
    # Get machine types from requests
    machine_types = get_machine_type_from_request(requests)
    
    # Initialize dictionary to store idle days for each machine type
    machine_idle_days = {}

    # Iterate through the idle days dictionary
    for request_id, days_late in idle_days.items():
        # Get the machine type for this request
        machine_type = machine_types.get(request_id)
        
        if machine_type:
            # Add the idle days to the corresponding machine type
            if machine_type in machine_idle_days:
                machine_idle_days[machine_type] += days_late
            else:
                machine_idle_days[machine_type] = days_late

    return machine_idle_days


machine_idle_days = get_idle_days_per_machine(idle_days, requests)

# Print the total idle days for each machine type
print("Total Idle Days for Each Machine Type:")
for machine_type, total_idle_days in machine_idle_days.items():
    print(f"{machine_type}: {total_idle_days} day(s)")


Total Idle Days for Each Machine Type:


In [1093]:
def calculate_idle_costs(machine_idle_days, penalty_costs_machines):
    """
    Calculate the costs of idle machine days.

    Args:
    - machine_idle_days: Dictionary where keys are machine types and values are the total number of idle days.
    - penalty_costs_machines: Dictionary where keys are machine types and values are the penalty costs per day.

    Returns:
    - idle_costs: Dictionary where keys are machine types and values are the total costs of idle days.
    """
    idle_costs = {}
    for machine_type, idle_days in machine_idle_days.items():
        penalty_cost = penalty_costs_machines.get(machine_type, 0)
        idle_costs[machine_type] = idle_days * penalty_cost
    return idle_costs

# Example usage:
idle_costs = calculate_idle_costs(machine_idle_days, penalty_costs_machines)

# Print the total costs of idle days for each machine type
print("Total Costs of Idle Days for Each Machine Type:")
total_idle_costs = 0
for machine_type, total_costs in idle_costs.items():
    print(f"{machine_type}: {total_costs} cost units")
    total_idle_costs += total_costs

print(f"Total Idle Costs: {total_idle_costs} cost ")


Total Costs of Idle Days for Each Machine Type:
Total Idle Costs: 0 cost 


In [1094]:
def format_assigned_requests(assigned_requests, current_day=None):
    formatted_output = ""
    
    # Group requests by day
    requests_by_day = {}
    for technician, requests in assigned_requests.items():
        for request, day in requests:
            requests_by_day.setdefault(day, []).append((technician, request))
    
    # If current_day is provided, filter to only that day
    if current_day is not None:
        sorted_days = [current_day]
    else:
        sorted_days = sorted(requests_by_day.keys())
    
    # Iterate through each day
    for day in sorted_days:
        formatted_output += f"DAY {day}\n"
        requests_on_day = requests_by_day.get(day, [])
        
        # Count number of technicians
        num_technicians = len(set(technician for technician, _ in requests_on_day))
        formatted_output += f"NUMBER_OF_TECHNICIANS = {num_technicians}\n"
        
        # Group requests by technician
        requests_by_technician = {}
        for technician, request in requests_on_day:
            requests_by_technician.setdefault(technician, []).append(request)
        
        # Format requests for each technician
        for technician, requests in requests_by_technician.items():
            formatted_output += f"{technician} {' '.join(str(request) for request in requests)}\n"
        
        formatted_output += "\n"  # Add a newline between days
    
    return formatted_output

# Example usage
formatted_output = format_assigned_requests(assigned_requests)
print(formatted_output)

DAY 3
NUMBER_OF_TECHNICIANS = 2
2 1
4 2

DAY 4
NUMBER_OF_TECHNICIANS = 3
4 3
5 4 6
1 5 8

DAY 5
NUMBER_OF_TECHNICIANS = 4
4 10
5 7
1 11
10 9 12

DAY 6
NUMBER_OF_TECHNICIANS = 1
6 13

DAY 7
NUMBER_OF_TECHNICIANS = 2
10 15
6 14

DAY 8
NUMBER_OF_TECHNICIANS = 3
5 16 18
1 17
7 19

DAY 9
NUMBER_OF_TECHNICIANS = 3
2 21
5 20
7 22

DAY 10
NUMBER_OF_TECHNICIANS = 3
4 23
10 25
8 24

DAY 11
NUMBER_OF_TECHNICIANS = 3
4 27
1 28 29
8 26

DAY 12
NUMBER_OF_TECHNICIANS = 1
4 30




In [1095]:
def store_technician_routes(assigned_requests):
    technician_routes = {}
    
    # Group requests by technician
    for technician, requests in assigned_requests.items():
        # Sort requests by day
        sorted_requests = sorted(requests, key=lambda x: x[1])
        
        # Initialize the route for the technician
        technician_routes[technician] = [[] for _ in range(max(day for _, day in sorted_requests) + 1)]
        
        # Fill in the route
        for request, day in sorted_requests:
            technician_routes[technician][day].append(request)
    
    return technician_routes

# Example usage
technician_routes = store_technician_routes(assigned_requests)

# Print the technician routes
for technician, route in technician_routes.items():
    print(f"Technician {technician} route:")
    for day, requests in enumerate(route):
        if requests:  # Check if the array is non-empty
            print(f"DAY {day}: {requests}")
    print()


Technician 2 route:
DAY 3: [1]
DAY 9: [21]

Technician 4 route:
DAY 3: [2]
DAY 4: [3]
DAY 5: [10]
DAY 10: [23]
DAY 11: [27]
DAY 12: [30]

Technician 5 route:
DAY 4: [4, 6]
DAY 5: [7]
DAY 8: [16, 18]
DAY 9: [20]

Technician 1 route:
DAY 4: [5, 8]
DAY 5: [11]
DAY 8: [17]
DAY 11: [28, 29]

Technician 10 route:
DAY 5: [9, 12]
DAY 7: [15]
DAY 10: [25]

Technician 6 route:
DAY 6: [13]
DAY 7: [14]

Technician 7 route:
DAY 8: [19]
DAY 9: [22]

Technician 8 route:
DAY 10: [24]
DAY 11: [26]



In [1096]:
def store_all_technician_routes(technician_routes):
    all_tech_routes = []
    for technician, route in technician_routes.items():
        for requests in route:
            all_tech_routes.extend([(technician, request) for request in requests])
    return all_tech_routes

# Example usage
all_tech_routes = store_all_technician_routes(technician_routes)

# Print all technician routes
print("All technician routes:")
print(all_tech_routes)


All technician routes:
[(2, 1), (2, 21), (4, 2), (4, 3), (4, 10), (4, 23), (4, 27), (4, 30), (5, 4), (5, 6), (5, 7), (5, 16), (5, 18), (5, 20), (1, 5), (1, 8), (1, 11), (1, 17), (1, 28), (1, 29), (10, 9), (10, 12), (10, 15), (10, 25), (6, 13), (6, 14), (7, 19), (7, 22), (8, 24), (8, 26)]


In [1097]:
def calculate_distances_for_all_technician_routes(all_tech_routes, distances):
    tech_route_distances = {}
    for tech, req in all_tech_routes:
        distance = distances[(tech, req)]
        if tech not in tech_route_distances:
            tech_route_distances[tech] = 0
        tech_route_distances[tech] += distance
    return tech_route_distances

# Example usage
tech_route_distances = calculate_distances_for_all_technician_routes(all_tech_routes, distances)

# Print technician route distances
print("Technician route distances:")
for technician, distance in tech_route_distances.items():
    print(f"Technician {technician} total distance: {distance}")
    
def calculate_total_distance_for_all_technicians(tech_route_distances):
    total_distance = sum(tech_route_distances.values())
    return total_distance

# Example usage
total_distance_all_techs = calculate_total_distance_for_all_technicians(tech_route_distances)

# Print the total distance for all technicians
print(f"Total distance for all technicians: {total_distance_all_techs}")



Technician route distances:
Technician 2 total distance: 394
Technician 4 total distance: 1971
Technician 5 total distance: 1878
Technician 1 total distance: 1051
Technician 10 total distance: 1188
Technician 6 total distance: 170
Technician 7 total distance: 537
Technician 8 total distance: 513
Total distance for all technicians: 7702


In [1098]:
def print_schedule_for_each_day(truck_routes, delivery_days):
    all_days = sorted(set(delivery_days.values()))  # Get all unique delivery days
    
    for current_day in all_days:
        print()  # Print an empty line before each day
        print(f"DAY = {current_day}")
        
        # Print truck routes for the current day
        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        
        if num_trucks_for_day > 0:
            print(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
            for truck_index, route in enumerate(truck_routes_for_day, start=1):
                print(truck_index, end=' ')
                for request_id in route:
                    print(request_id, end=' ')  # Print each request ID in the route
                print()  # Newline after each truck route
        else:
            print("NUMBER_OF_TRUCKS = 0")

# Example usage
print("Truck Routes with Depot:")
print_schedule_for_each_day(truck_routes_extracted, delivery_days)


Truck Routes with Depot:

DAY = 2
NUMBER_OF_TRUCKS = 3
1 1 
2 2 
3 3 

DAY = 3
NUMBER_OF_TRUCKS = 5
1 4 
2 5 
3 6 
4 7 
5 8 

DAY = 4
NUMBER_OF_TRUCKS = 4
1 9 
2 10 
3 11 
4 12 

DAY = 5
NUMBER_OF_TRUCKS = 1
1 13 

DAY = 6
NUMBER_OF_TRUCKS = 2
1 14 
2 15 

DAY = 7
NUMBER_OF_TRUCKS = 3
1 16 
2 17 18 
3 19 

DAY = 8
NUMBER_OF_TRUCKS = 3
1 20 
2 21 
3 22 

DAY = 9
NUMBER_OF_TRUCKS = 4
1 23 
2 24 
3 25 
4 26 

DAY = 10
NUMBER_OF_TRUCKS = 3
1 27 
2 28 
3 29 

DAY = 11
NUMBER_OF_TRUCKS = 1
1 30 


In [1099]:
def print_schedule(truck_routes, delivery_days, assigned_requests, title, total_truck_distance, total_distance_all_techs):
    IDLE_MACHINE_COST = 0         # Define the actual cost
    
    # Calculate additional statistics
    days_with_trucks = set(day for _, day in truck_routes)
    days_with_technicians = set(day for day in delivery_days.values())
    all_days = sorted(set(days_with_trucks) | set(days_with_technicians))  # Union of days with trucks and technician days
    
    max_trucks_used = max(sum(1 for route, day in truck_routes if day == current_day) for current_day in days_with_trucks)
    max_technicians_used = max(sum(1 for day in delivery_days.values() if day == current_day) for current_day in days_with_technicians)
    
    number_of_truck_days = len(days_with_trucks)
    number_of_technician_days = len(days_with_technicians)

    # Calculate costs
    total_truck_distance_cost = total_truck_distance * TRUCK_DISTANCE_COST
    total_truck_day_cost = number_of_truck_days * TRUCK_DAY_COST
    total_truck_cost = max_trucks_used * TRUCK_COST
    total_technician_distance_cost = total_distance_all_techs * TECHNICIAN_DISTANCE_COST
    total_technician_day_cost = number_of_technician_days * TECHNICIAN_DAY_COST
    total_technician_cost = max_technicians_used * TECHNICIAN_COST

    total_cost = (total_truck_distance_cost + total_truck_day_cost + total_truck_cost +
                  total_technician_distance_cost + total_technician_day_cost + total_technician_cost + IDLE_MACHINE_COST)

    # Print the additional information
    print("DATASET = ORTEC Caroline VeRoLog 2019")
    print(f"NAME = {title}")
    print(f"TRUCK_DISTANCE = {int(total_truck_distance)}")
    print(f"NUMBER_OF_TRUCK_DAYS = {number_of_truck_days}")
    print(f"NUMBER_OF_TRUCKS_USED = {max_trucks_used}")
    print(f"TECHNICIAN_DISTANCE = {int(total_distance_all_techs)}")
    print(f"NUMBER_OF_TECHNICIAN_DAYS = {number_of_technician_days}")
    print(f"NUMBER_OF_TECHNICIANS_USED = {max_technicians_used}")
    print(f"IDLE_MACHINE_COSTS = {IDLE_MACHINE_COST}")
    print(f"TOTAL_COST = {int(total_cost)}")
    print()

    # Print the schedule for each day
    for current_day in all_days:
        print()  # Print an empty line before each day
        print(f"DAY = {current_day}")
        
        # Print truck routes for the current day
        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        print(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
        for truck_index, route in enumerate(truck_routes_for_day, start=1):
            print(truck_index, end=' ')
            for request_id in route:
                print(request_id, end=' ')  # Print each request ID in the route
            print()  # Newline after each truck route
        
        # Print technician assignments for the current day
        formatted_output = format_assigned_requests(assigned_requests, current_day)
        # Remove the DAY line from the formatted output
        formatted_output = '\n'.join(formatted_output.split('\n')[1:])
        print(formatted_output)

# Example usage
print_schedule(truck_routes_extracted, delivery_days, assigned_requests, title, total_truck_distance, total_distance_all_techs)


DATASET = ORTEC Caroline VeRoLog 2019
NAME = Instance 19
TRUCK_DISTANCE = 16819
NUMBER_OF_TRUCK_DAYS = 10
NUMBER_OF_TRUCKS_USED = 5
TECHNICIAN_DISTANCE = 7702
NUMBER_OF_TECHNICIAN_DAYS = 10
NUMBER_OF_TECHNICIANS_USED = 5
IDLE_MACHINE_COSTS = 0
TOTAL_COST = 813202


DAY = 2
NUMBER_OF_TRUCKS = 3
1 1 
2 2 
3 3 
NUMBER_OF_TECHNICIANS = 0



DAY = 3
NUMBER_OF_TRUCKS = 5
1 4 
2 5 
3 6 
4 7 
5 8 
NUMBER_OF_TECHNICIANS = 2
2 1
4 2



DAY = 4
NUMBER_OF_TRUCKS = 4
1 9 
2 10 
3 11 
4 12 
NUMBER_OF_TECHNICIANS = 3
4 3
5 4 6
1 5 8



DAY = 5
NUMBER_OF_TRUCKS = 1
1 13 
NUMBER_OF_TECHNICIANS = 4
4 10
5 7
1 11
10 9 12



DAY = 6
NUMBER_OF_TRUCKS = 2
1 14 
2 15 
NUMBER_OF_TECHNICIANS = 1
6 13



DAY = 7
NUMBER_OF_TRUCKS = 3
1 16 
2 17 18 
3 19 
NUMBER_OF_TECHNICIANS = 2
10 15
6 14



DAY = 8
NUMBER_OF_TRUCKS = 3
1 20 
2 21 
3 22 
NUMBER_OF_TECHNICIANS = 3
5 16 18
1 17
7 19



DAY = 9
NUMBER_OF_TRUCKS = 4
1 23 
2 24 
3 25 
4 26 
NUMBER_OF_TECHNICIANS = 3
2 21
5 20
7 22



DAY = 10
NUMBER_OF_TRUCKS = 3
1

In [1100]:
print(title)

Instance 19


In [1101]:
def print_schedule(truck_routes, delivery_days, assigned_requests):
    all_days = set(delivery_days.values())  # Get all unique delivery days from trucks
    
    # Get all unique days from technician assignments
    for requests in assigned_requests.values():
        for _, day in requests:
            all_days.add(day)
    
    all_days = sorted(all_days)  # Sort all unique days
    
    for current_day in all_days:
        print()  # Print an empty line before each day
        print(f"DAY = {current_day}")
        
        # Print truck routes for the current day
        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        print(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
        for truck_index, route in enumerate(truck_routes_for_day, start=1):
            print(truck_index, end=' ')
            for request_id in route:
                print(request_id, end=' ')  # Print each request ID in the route
            print()  # Newline after each truck route
        
        # Print technician assignments for the current day
        formatted_output = format_assigned_requests(assigned_requests, current_day)
        # Remove the DAY line from the formatted output
        formatted_output = '\n'.join(formatted_output.split('\n')[1:])
        print(formatted_output)

# Example usage
print_schedule(truck_routes_extracted, delivery_days, assigned_requests)



DAY = 2
NUMBER_OF_TRUCKS = 3
1 1 
2 2 
3 3 
NUMBER_OF_TECHNICIANS = 0



DAY = 3
NUMBER_OF_TRUCKS = 5
1 4 
2 5 
3 6 
4 7 
5 8 
NUMBER_OF_TECHNICIANS = 2
2 1
4 2



DAY = 4
NUMBER_OF_TRUCKS = 4
1 9 
2 10 
3 11 
4 12 
NUMBER_OF_TECHNICIANS = 3
4 3
5 4 6
1 5 8



DAY = 5
NUMBER_OF_TRUCKS = 1
1 13 
NUMBER_OF_TECHNICIANS = 4
4 10
5 7
1 11
10 9 12



DAY = 6
NUMBER_OF_TRUCKS = 2
1 14 
2 15 
NUMBER_OF_TECHNICIANS = 1
6 13



DAY = 7
NUMBER_OF_TRUCKS = 3
1 16 
2 17 18 
3 19 
NUMBER_OF_TECHNICIANS = 2
10 15
6 14



DAY = 8
NUMBER_OF_TRUCKS = 3
1 20 
2 21 
3 22 
NUMBER_OF_TECHNICIANS = 3
5 16 18
1 17
7 19



DAY = 9
NUMBER_OF_TRUCKS = 4
1 23 
2 24 
3 25 
4 26 
NUMBER_OF_TECHNICIANS = 3
2 21
5 20
7 22



DAY = 10
NUMBER_OF_TRUCKS = 3
1 27 
2 28 
3 29 
NUMBER_OF_TECHNICIANS = 3
4 23
10 25
8 24



DAY = 11
NUMBER_OF_TRUCKS = 1
1 30 
NUMBER_OF_TECHNICIANS = 3
4 27
1 28 29
8 26



DAY = 12
NUMBER_OF_TRUCKS = 0
NUMBER_OF_TECHNICIANS = 1
4 30




In [1102]:
def print_schedule(truck_routes, delivery_days, assigned_requests, total_truck_distance, total_distance_all_techs):
    all_days = set(delivery_days.values())  # Get all unique delivery days from trucks

    # Get all unique days from technician assignments
    for requests in assigned_requests.values():
        for _, day in requests:
            all_days.add(day)

    all_days = sorted(all_days)  # Sort all unique days
    total_days = max(all_days) if all_days else 0  # Find the highest day number

    max_trucks_used = 0
    max_technicians_used = 0
    number_of_truck_days = 0
    number_of_technician_days = 0

    for current_day in range(1, total_days + 1):
        num_trucks_for_day = 0
        num_technicians_for_day = 0

        print()  # Print an empty line before each day
        print(f"DAY = {current_day}")

        # Print truck routes for the current day
        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        if num_trucks_for_day > 0:
            number_of_truck_days += 1
        if num_trucks_for_day > max_trucks_used:
            max_trucks_used = num_trucks_for_day
        print(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
        for truck_index, route in enumerate(truck_routes_for_day, start=1):
            print(truck_index, end=' ')
            for request_id in route:
                print(request_id, end=' ')  # Print each request ID in the route
            print()  # Newline after each truck route

        # Print technician assignments for the current day
        technician_requests_for_day = {tech: reqs for tech, reqs in assigned_requests.items() if any(day == current_day for _, day in reqs)}
        num_technicians_for_day = len(technician_requests_for_day)
        if num_technicians_for_day > 0:
            number_of_technician_days += 1
            if num_technicians_for_day > max_technicians_used:
                max_technicians_used = num_technicians_for_day
        print(f"NUMBER_OF_TECHNICIANS = {num_technicians_for_day}")
        for tech_id, requests in technician_requests_for_day.items():
            tech_requests_for_day = [req for req, day in requests if day == current_day]
            if tech_requests_for_day:
                print(f"{tech_id} " + ' '.join(map(str, tech_requests_for_day)))

    IDLE_MACHINE_COST = total_idle_costs

    total_truck_distance_cost = total_truck_distance * TRUCK_DISTANCE_COST
    total_truck_day_cost = number_of_truck_days * TRUCK_DAY_COST
    total_truck_cost = max_trucks_used * TRUCK_COST
    total_technician_distance_cost = total_distance_all_techs * TECHNICIAN_DISTANCE_COST
    total_technician_day_cost = number_of_technician_days * TECHNICIAN_DAY_COST
    total_technician_cost = max_technicians_used * TECHNICIAN_COST

    total_cost = (total_truck_distance_cost + total_truck_day_cost + total_truck_cost +
                  total_technician_distance_cost + total_technician_day_cost + total_technician_cost + IDLE_MACHINE_COST)

    return (number_of_truck_days, number_of_technician_days, max_trucks_used, max_technicians_used, total_cost)

# Example usage
number_of_truck_days, number_of_technician_days, max_trucks_used, max_technicians_used, total_cost = print_schedule(truck_routes_extracted, delivery_days, assigned_requests, total_truck_distance, total_distance_all_techs)

# Now you have access to these calculated values outside the function scope
print(f"\n")
print(f"TRUCK_DISTANCE = {int(total_truck_distance)}")
print(f"NUMBER_OF_TRUCK_DAYS = {number_of_truck_days}")
print(f"NUMBER_OF_TECHNICIAN_DAYS = {number_of_technician_days}")
print(f"TECHNICIAN_DISTANCE = {int(total_distance_all_techs)}")
print(f"NUMBER_OF_TRUCKS_USED = {max_trucks_used}")
print(f"NUMBER_OF_TECHNICIANS_USED = {max_technicians_used}")
print(f"TOTAL_COST = {int(total_cost)}")



DAY = 1
NUMBER_OF_TRUCKS = 0
NUMBER_OF_TECHNICIANS = 0

DAY = 2
NUMBER_OF_TRUCKS = 3
1 1 
2 2 
3 3 
NUMBER_OF_TECHNICIANS = 0

DAY = 3
NUMBER_OF_TRUCKS = 5
1 4 
2 5 
3 6 
4 7 
5 8 
NUMBER_OF_TECHNICIANS = 2
2 1
4 2

DAY = 4
NUMBER_OF_TRUCKS = 4
1 9 
2 10 
3 11 
4 12 
NUMBER_OF_TECHNICIANS = 3
4 3
5 4 6
1 5 8

DAY = 5
NUMBER_OF_TRUCKS = 1
1 13 
NUMBER_OF_TECHNICIANS = 4
4 10
5 7
1 11
10 9 12

DAY = 6
NUMBER_OF_TRUCKS = 2
1 14 
2 15 
NUMBER_OF_TECHNICIANS = 1
6 13

DAY = 7
NUMBER_OF_TRUCKS = 3
1 16 
2 17 18 
3 19 
NUMBER_OF_TECHNICIANS = 2
10 15
6 14

DAY = 8
NUMBER_OF_TRUCKS = 3
1 20 
2 21 
3 22 
NUMBER_OF_TECHNICIANS = 3
5 16 18
1 17
7 19

DAY = 9
NUMBER_OF_TRUCKS = 4
1 23 
2 24 
3 25 
4 26 
NUMBER_OF_TECHNICIANS = 3
2 21
5 20
7 22

DAY = 10
NUMBER_OF_TRUCKS = 3
1 27 
2 28 
3 29 
NUMBER_OF_TECHNICIANS = 3
4 23
10 25
8 24

DAY = 11
NUMBER_OF_TRUCKS = 1
1 30 
NUMBER_OF_TECHNICIANS = 3
4 27
1 28 29
8 26

DAY = 12
NUMBER_OF_TRUCKS = 0
NUMBER_OF_TECHNICIANS = 1
4 30


TRUCK_DISTANCE = 1681

In [1103]:
def final_print_schedule(title, truck_routes, delivery_days, assigned_requests, total_truck_distance, total_distance_all_techs):
    all_days = set(delivery_days.values())  # Get all unique delivery days from trucks
    max_trucks_used = 0
    max_technicians_used = 0
    number_of_truck_days = 0
    number_of_technician_days = 0

    # Get all unique days from technician assignments
    for requests in assigned_requests.values():
        for _, day in requests:
            all_days.add(day)

    all_days = sorted(all_days)  # Sort all unique days

    # Initialize formatted day-by-day output
    daily_output = []

    for current_day in all_days:
        num_trucks_for_day = 0
        num_technicians_for_day = 0

        day_output = []
        day_output.append(f"DAY = {current_day}")

        # Print truck routes for the current day
        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        if num_trucks_for_day > 0:
            number_of_truck_days += 1
        if num_trucks_for_day > max_trucks_used:
            max_trucks_used = num_trucks_for_day
        day_output.append(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
        for truck_index, route in enumerate(truck_routes_for_day, start=1):
            route_output = f"{truck_index} " + ' '.join(map(str, route))
            day_output.append(route_output)

        # Print technician assignments for the current day
        formatted_output = format_assigned_requests(assigned_requests, current_day)
        num_technicians_for_day = formatted_output.count('NUMBER_OF_TECHNICIANS = ')
        if num_technicians_for_day > 0:
            technician_info = formatted_output.split('NUMBER_OF_TECHNICIANS = ')[1]
            num_technicians = int(technician_info.split()[0])
            if num_technicians > 0:
                number_of_technician_days += 1
                if num_technicians > max_technicians_used:
                    max_technicians_used = num_technicians
        if num_technicians_for_day > max_technicians_used:
            max_technicians_used = num_technicians_for_day
        # Remove the DAY line from the formatted output
        formatted_output = '\n'.join(formatted_output.split('\n')[1:])
        day_output.append(formatted_output)

        daily_output.append('\n'.join(day_output))

    # Print summary information
    IDLE_MACHINE_COST = total_idle_costs
    total_truck_distance_cost = total_truck_distance * TRUCK_DISTANCE_COST
    total_truck_day_cost = number_of_truck_days * TRUCK_DAY_COST
    total_truck_cost = max_trucks_used * TRUCK_COST
    total_technician_distance_cost = total_distance_all_techs * TECHNICIAN_DISTANCE_COST
    total_technician_day_cost = number_of_technician_days * TECHNICIAN_DAY_COST
    total_technician_cost = max_technicians_used * TECHNICIAN_COST

    total_cost = (total_truck_distance_cost + total_truck_day_cost + total_truck_cost +
                  total_technician_distance_cost + total_technician_day_cost + total_technician_cost + IDLE_MACHINE_COST)

    print(f"DATASET = ORTEC Caroline VeRoLog 2019")
    print(f"NAME = {title}")
    print(f"TRUCK_DISTANCE = {int(total_truck_distance)}")
    print(f"NUMBER_OF_TRUCK_DAYS = {number_of_truck_days}")
    print(f"NUMBER_OF_TECHNICIAN_DAYS = {number_of_technician_days}")
    print(f"TECHNICIAN_DISTANCE = {int(total_distance_all_techs)}")
    print(f"NUMBER_OF_TRUCKS_USED = {max_trucks_used}")
    print(f"NUMBER_OF_TECHNICIANS_USED = {max_technicians_used}")
    print(f"IDLE_MACHINE_COSTS = {total_idle_costs}")
    print(f"TOTAL_COST = {int(total_cost)}")

    # Print the day-by-day output
    for output in daily_output:
        print()
        print(output)

    return (number_of_truck_days, number_of_technician_days, max_trucks_used, max_technicians_used, total_cost)

# Example usage
number_of_truck_days, number_of_technician_days, max_trucks_used, max_technicians_used, total_cost = final_print_schedule(title, truck_routes_extracted, delivery_days, assigned_requests, total_truck_distance, total_distance_all_techs)


DATASET = ORTEC Caroline VeRoLog 2019
NAME = Instance 19
TRUCK_DISTANCE = 16819
NUMBER_OF_TRUCK_DAYS = 10
NUMBER_OF_TECHNICIAN_DAYS = 10
TECHNICIAN_DISTANCE = 7702
NUMBER_OF_TRUCKS_USED = 5
NUMBER_OF_TECHNICIANS_USED = 4
IDLE_MACHINE_COSTS = 0
TOTAL_COST = 793202

DAY = 2
NUMBER_OF_TRUCKS = 3
1 1
2 2
3 3
NUMBER_OF_TECHNICIANS = 0



DAY = 3
NUMBER_OF_TRUCKS = 5
1 4
2 5
3 6
4 7
5 8
NUMBER_OF_TECHNICIANS = 2
2 1
4 2



DAY = 4
NUMBER_OF_TRUCKS = 4
1 9
2 10
3 11
4 12
NUMBER_OF_TECHNICIANS = 3
4 3
5 4 6
1 5 8



DAY = 5
NUMBER_OF_TRUCKS = 1
1 13
NUMBER_OF_TECHNICIANS = 4
4 10
5 7
1 11
10 9 12



DAY = 6
NUMBER_OF_TRUCKS = 2
1 14
2 15
NUMBER_OF_TECHNICIANS = 1
6 13



DAY = 7
NUMBER_OF_TRUCKS = 3
1 16
2 17 18
3 19
NUMBER_OF_TECHNICIANS = 2
10 15
6 14



DAY = 8
NUMBER_OF_TRUCKS = 3
1 20
2 21
3 22
NUMBER_OF_TECHNICIANS = 3
5 16 18
1 17
7 19



DAY = 9
NUMBER_OF_TRUCKS = 4
1 23
2 24
3 25
4 26
NUMBER_OF_TECHNICIANS = 3
2 21
5 20
7 22



DAY = 10
NUMBER_OF_TRUCKS = 3
1 27
2 28
3 29
NUMBER_OF_TE

In [1104]:
def print_schedule(truck_routes, delivery_days, assigned_requests, total_truck_distance, total_distance_all_techs, output_file_name):
    all_days = set(delivery_days.values())  # Get all unique delivery days from trucks

    # Get all unique days from technician assignments
    for requests in assigned_requests.values():
        for _, day in requests:
            all_days.add(day)

    all_days = sorted(all_days)  # Sort all unique days
    total_days = max(all_days) if all_days else 0  # Find the highest day number

    max_trucks_used = 0
    max_technicians_used = 0
    number_of_truck_days = 0
    number_of_technician_days = 0

    daily_output = []

    for current_day in range(1, DAYS + 1):
        num_trucks_for_day = 0
        num_technicians_for_day = 0

        day_output = []  # Stores the output for the current day

        day_output.append(f"DAY = {current_day}")

        # Print truck routes for the current day
        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        if num_trucks_for_day > 0:
            number_of_truck_days += 1
        if num_trucks_for_day > max_trucks_used:
            max_trucks_used = num_trucks_for_day
        day_output.append(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
        for truck_index, route in enumerate(truck_routes_for_day, start=1):
            route_output = f"{truck_index} " + ' '.join(map(str, route))
            day_output.append(route_output)

        # Print technician assignments for the current day
        technician_requests_for_day = {tech: reqs for tech, reqs in assigned_requests.items() if any(day == current_day for _, day in reqs)}
        num_technicians_for_day = len(technician_requests_for_day)
        if num_technicians_for_day > 0:
            number_of_technician_days += 1
            if num_technicians_for_day > max_technicians_used:
                max_technicians_used = num_technicians_for_day
        day_output.append(f"NUMBER_OF_TECHNICIANS = {num_technicians_for_day}")
        for tech_id, requests in technician_requests_for_day.items():
            tech_requests_for_day = [req for req, day in requests if day == current_day]
            if tech_requests_for_day:
                day_output.append(f"{tech_id} " + ' '.join(map(str, tech_requests_for_day)))

        daily_output.append('\n'.join(day_output))

    # Calculate costs
    IDLE_MACHINE_COST = total_idle_costs

    total_truck_distance_cost = total_truck_distance * TRUCK_DISTANCE_COST
    total_truck_day_cost = number_of_truck_days * TRUCK_DAY_COST
    total_truck_cost = max_trucks_used * TRUCK_COST
    total_technician_distance_cost = total_distance_all_techs * TECHNICIAN_DISTANCE_COST
    total_technician_day_cost = number_of_technician_days * TECHNICIAN_DAY_COST
    total_technician_cost = max_technicians_used * TECHNICIAN_COST

    total_cost = (total_truck_distance_cost + total_truck_day_cost + total_truck_cost +
                  total_technician_distance_cost + total_technician_day_cost + total_technician_cost + IDLE_MACHINE_COST)

    # Open a file to write the output
    with open(output_file_name, "w") as file:
        # Write the summary information
        file.write(f"DATASET = ORTEC Caroline VeRoLog 2019\n")
        file.write(f"NAME = {title}\n")
        file.write(f"TRUCK_DISTANCE = {int(total_truck_distance)}\n")
        file.write(f"NUMBER_OF_TRUCK_DAYS = {number_of_truck_days}\n")
        file.write(f"NUMBER_OF_TECHNICIAN_DAYS = {number_of_technician_days}\n")
        file.write(f"TECHNICIAN_DISTANCE = {int(total_distance_all_techs)}\n")
        file.write(f"NUMBER_OF_TRUCKS_USED = {max_trucks_used}\n")
        file.write(f"NUMBER_OF_TECHNICIANS_USED = {max_technicians_used}\n")
        file.write(f"IDLE_MACHINE_COSTS = {IDLE_MACHINE_COST}\n")
        file.write(f"TOTAL_COST = {int(total_cost)}\n")

        # Write the day-by-day output
        for output in daily_output:
            file.write("\n")
            file.write(output)
            file.write("\n")

    return (number_of_truck_days, number_of_technician_days, max_trucks_used, max_technicians_used, total_cost)


# Example usage
number_of_truck_days, number_of_technician_days, max_trucks_used, max_technicians_used, total_cost = print_schedule(truck_routes_extracted, delivery_days, assigned_requests, total_truck_distance, total_distance_all_techs, output_file_name)

# Now you have access to these calculated values outside the function scope
print(f"TRUCK_DISTANCE = {int(total_truck_distance)}")
print(f"NUMBER_OF_TRUCK_DAYS = {number_of_truck_days}")
print(f"NUMBER_OF_TECHNICIAN_DAYS = {number_of_technician_days}")
print(f"TECHNICIAN_DISTANCE = {int(total_distance_all_techs)}")
print(f"NUMBER_OF_TRUCKS_USED = {max_trucks_used}")
print(f"NUMBER_OF_TECHNICIANS_USED = {max_technicians_used}")
print(f"TOTAL_COST = {int(total_cost)}")


TRUCK_DISTANCE = 16819
NUMBER_OF_TRUCK_DAYS = 10
NUMBER_OF_TECHNICIAN_DAYS = 10
TECHNICIAN_DISTANCE = 7702
NUMBER_OF_TRUCKS_USED = 5
NUMBER_OF_TECHNICIANS_USED = 4
TOTAL_COST = 793202
